# Importing libraries

In [1]:
import pypots
import os
import sys
from pypots.nn.functional import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012
from missingData.toolkits import toolkits

2025-03-16 19:16:31.891051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742163391.906517  107374 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742163391.911252  107374 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-16 19:16:31.927745: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/pypots/nn/functional/cuda.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



# Loading database

### StandardScaler

In [2]:
set_random_seed()
physionet2012_dataset_standard = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)
print(physionet2012_dataset_standard.keys())

2025-03-16 19:16:37 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-03-16 19:16:37 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-03-16 19:16:37 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-03-16 19:16:37 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-03-16 19:16:37 [INFO]: Loaded successfully!
2025-03-16 19:16:51 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-16 19:16:51 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-16 19:16:51 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
scaler_standard = physionet2012_dataset_standard["scaler"]

In [4]:
dataset_for_training_standard, dataset_for_validating_standard, dataset_for_testing_ori_standard, dataset_for_testing_standard = toolkits.separating_dataset(physionet2012_dataset_standard)

### MinMaxScaler

In [5]:
set_random_seed()
physionet2012_dataset_minmax = preprocess_physionet2012(subset="all", rate=0.1, normalization=2)
print(physionet2012_dataset_minmax.keys())

2025-03-16 17:45:27 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-03-16 17:45:27 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-03-16 17:45:27 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-03-16 17:45:27 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-03-16 17:45:27 [INFO]: Loaded successfully!
2025-03-16 17:45:40 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-16 17:45:40 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-16 17:45:40 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [6]:
scaler_minmax = physionet2012_dataset_minmax["scaler"]

In [7]:
dataset_for_training_minmax, dataset_for_validating_minmax, dataset_for_testing_minmax, dataset_for_testing_ori_minmax = toolkits.separating_dataset(physionet2012_dataset_minmax)

# Creating indicating mask

### StandardScaler

In [8]:
dataset_for_testing_ori_standard_norm = toolkits.dict_to_list(dataset_for_testing_ori_standard)
dataset_for_testing_standard_norm = toolkits.dict_to_list(dataset_for_testing_standard)
indicating_mask_variable_standard_norm, test_X_ori_variable_standard_norm = toolkits.components_mae(dataset_for_testing_ori_standard_norm, dataset_for_testing_standard_norm)
indicating_mask_variable_standard_norm = toolkits.pre_reshape(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.pre_reshape(test_X_ori_variable_standard_norm)
indicating_mask_variable_standard_norm = toolkits.reshape_variable(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.reshape_variable(test_X_ori_variable_standard_norm)

In [9]:
dataset_for_testing_ori_standard_des = toolkits.dict_to_list(dataset_for_testing_ori_standard)
dataset_for_testing_standard_des = toolkits.dict_to_list(dataset_for_testing_standard)
dataset_for_testing_ori_standard_des = toolkits.pre_reshape(dataset_for_testing_ori_standard_des)
dataset_for_testing_standard_des = toolkits.pre_reshape(dataset_for_testing_standard_des)
dataset_for_testing_ori_standard_des = toolkits.desnormalization(dataset_for_testing_ori_standard_des, scaler_standard)
dataset_for_testing_standard_des = toolkits.desnormalization(dataset_for_testing_standard_des, scaler_standard)
indicating_mask_variable_standard_des, test_X_ori_variable_standard_des = toolkits.components_mae(dataset_for_testing_ori_standard_des, dataset_for_testing_standard_des)
indicating_mask_variable_standard_des = toolkits.reshape_variable(indicating_mask_variable_standard_des)
test_X_ori_variable_standard_des = toolkits.reshape_variable(test_X_ori_variable_standard_des)

### MinMaxScaler 

In [10]:
dataset_for_testing_ori_minmax_norm = toolkits.dict_to_list(dataset_for_testing_ori_minmax)
dataset_for_testing_minmax_norm = toolkits.dict_to_list(dataset_for_testing_minmax)
indicating_mask_variable_minmax_norm, test_X_ori_variable_minmax_norm = toolkits.components_mae(dataset_for_testing_ori_minmax_norm, dataset_for_testing_minmax_norm)
indicating_mask_variable_minmax_norm = toolkits.pre_reshape(indicating_mask_variable_minmax_norm)
test_X_ori_variable_minmax_norm = toolkits.pre_reshape(test_X_ori_variable_minmax_norm)
indicating_mask_variable_minmax_norm = toolkits.reshape_variable(indicating_mask_variable_minmax_norm)
test_X_ori_variable_minmax_norm = toolkits.reshape_variable(test_X_ori_variable_minmax_norm)

In [11]:
dataset_for_testing_ori_minmax_des = toolkits.dict_to_list(dataset_for_testing_ori_minmax)
dataset_for_testing_minmax_des = toolkits.dict_to_list(dataset_for_testing_minmax)
dataset_for_testing_ori_minmax_des = toolkits.pre_reshape(dataset_for_testing_ori_minmax_des)
dataset_for_testing_minmax_des = toolkits.pre_reshape(dataset_for_testing_minmax_des)
dataset_for_testing_ori_minmax_des = toolkits.desnormalization(dataset_for_testing_ori_minmax_des, scaler_minmax)
dataset_for_testing_minmax_des = toolkits.desnormalization(dataset_for_testing_minmax_des, scaler_minmax)
indicating_mask_variable_minmax_des, test_X_ori_variable_minmax_des = toolkits.components_mae(dataset_for_testing_ori_minmax_des, dataset_for_testing_minmax_des)
indicating_mask_variable_minmax_des = toolkits.reshape_variable(indicating_mask_variable_minmax_des)
test_X_ori_variable_minmax_des = toolkits.reshape_variable(test_X_ori_variable_minmax_des)

# Inicialize the models

### SAITS

<h4> Inicialize new model (Standard)</h4>

In [12]:
saits_standard = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 15:20:30 [INFO]: No given device, using default device: cpu
2025-03-16 15:20:30 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250316_T152030
2025-03-16 15:20:30 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250316_T152030/tensorboard
2025-03-16 15:20:30 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize existing model (Standard)</h4>

In [12]:
saits_standard = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 17:46:50 [INFO]: No given device, using default device: cpu
2025-03-16 17:46:50 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 17:46:50 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize new model (MinMax)</h4>

In [13]:
saits_minmax = SAITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 15:20:40 [INFO]: No given device, using default device: cpu
2025-03-16 15:20:40 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250316_T152040
2025-03-16 15:20:40 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250316_T152040/tensorboard
2025-03-16 15:20:40 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize existing model (MinMax)</h4>

In [13]:
saits_minmax = SAITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 17:46:55 [INFO]: No given device, using default device: cpu
2025-03-16 17:46:55 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 17:46:55 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


### BRITS

<h4> Inicialize new model (Standard)</h4>

In [19]:
brits_standard = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 15:29:19 [INFO]: No given device, using default device: cpu
2025-03-16 15:29:19 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250316_T152919
2025-03-16 15:29:19 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250316_T152919/tensorboard
2025-03-16 15:29:19 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize existing model (Standard)</h4>

In [14]:
brits_standard = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 17:47:00 [INFO]: No given device, using default device: cpu
2025-03-16 17:47:00 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 17:47:00 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize new model (MinMax)</h4>

In [20]:
brits_minmax = BRITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 15:29:25 [INFO]: No given device, using default device: cpu
2025-03-16 15:29:25 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250316_T152925
2025-03-16 15:29:25 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250316_T152925/tensorboard
2025-03-16 15:29:25 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize existing model (MinMax)</h4>

In [15]:
brits_minmax = BRITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 17:47:04 [INFO]: No given device, using default device: cpu
2025-03-16 17:47:04 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 17:47:04 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


### USGAN

<h4> Inicialize new model (Standard)</h4>

In [23]:
us_gan_standard = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 15:45:59 [INFO]: No given device, using default device: cpu
2025-03-16 15:45:59 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250316_T154559
2025-03-16 15:45:59 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250316_T154559/tensorboard
2025-03-16 15:45:59 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize existing model (Standard)</h4>

In [16]:
us_gan_standard = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 17:47:08 [INFO]: No given device, using default device: cpu
2025-03-16 17:47:08 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 17:47:08 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize new model (MinMax)</h4>

In [24]:
us_gan_minmax = USGAN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 15:46:05 [INFO]: No given device, using default device: cpu
2025-03-16 15:46:05 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250316_T154605
2025-03-16 15:46:05 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250316_T154605/tensorboard
2025-03-16 15:46:05 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize existing model (MinMax)</h4>

In [17]:
us_gan_minmax = USGAN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 17:47:13 [INFO]: No given device, using default device: cpu
2025-03-16 17:47:13 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 17:47:13 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


### GPVAE

<h4> Inicialize new model (Standard)</h4>

In [27]:
gp_vae_standard = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 16:16:53 [INFO]: No given device, using default device: cpu
2025-03-16 16:16:53 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250316_T161653
2025-03-16 16:16:53 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250316_T161653/tensorboard
2025-03-16 16:16:53 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize existing model (Standard)</h4>

In [18]:
gp_vae_standard = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 17:47:19 [INFO]: No given device, using default device: cpu
2025-03-16 17:47:19 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 17:47:19 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize new model (MinMax)</h4>

In [28]:
gp_vae_minmax = GPVAE(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 16:17:44 [INFO]: No given device, using default device: cpu
2025-03-16 16:17:44 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250316_T161744
2025-03-16 16:17:44 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250316_T161744/tensorboard
2025-03-16 16:17:44 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize existing model (MinMax)</h4>

In [19]:
gp_vae_minmax= GPVAE(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 17:47:25 [INFO]: No given device, using default device: cpu
2025-03-16 17:47:25 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 17:47:25 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


### MRNN

<h4> Inicialize new model (Standard)</h4>

In [31]:
mrnn_standard = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 16:27:03 [INFO]: No given device, using default device: cpu
2025-03-16 16:27:03 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250316_T162703
2025-03-16 16:27:03 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250316_T162703/tensorboard
2025-03-16 16:27:03 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize existing model (Standard)</h4>

In [20]:
mrnn_standard = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 17:47:30 [INFO]: No given device, using default device: cpu
2025-03-16 17:47:30 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 17:47:30 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize new model (MinMax)</h4>

In [32]:
mrnn_minmax = MRNN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 16:27:08 [INFO]: No given device, using default device: cpu
2025-03-16 16:27:08 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250316_T162708
2025-03-16 16:27:08 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250316_T162708/tensorboard
2025-03-16 16:27:08 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize existing model (MinMax)</h4>

In [21]:
mrnn_minmax = MRNN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 17:47:33 [INFO]: No given device, using default device: cpu
2025-03-16 17:47:33 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 17:47:33 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


# Train/Load the models

### SAITS

<h4>Train - Standard Scaler </h4>

In [14]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
saits_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-16 15:21:05 [INFO]: Epoch 001 - training loss (MSE): 0.8186, validation MSE: 6.7439
2025-03-16 15:21:15 [INFO]: Epoch 002 - training loss (MSE): 0.5314, validation MSE: 6.6840
2025-03-16 15:21:25 [INFO]: Epoch 003 - training loss (MSE): 0.5029, validation MSE: 6.6729
2025-03-16 15:21:35 [INFO]: Epoch 004 - training loss (MSE): 0.4546, validation MSE: 6.6573
2025-03-16 15:21:44 [INFO]: Epoch 005 - training loss (MSE): 0.4356, validation MSE: 6.6584
2025-03-16 15:21:54 [INFO]: Epoch 006 - training loss (MSE): 0.4727, validation MSE: 6.6585
2025-03-16 15:22:04 [INFO]: Epoch 007 - training loss (MSE): 0.4030, validation MSE: 6.6770
2025-03-16 15:22:04 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-03-16 15:22:04 [INFO]: Finished training. The best model is from epoch#4.
2025-03-16 15:22:04 [INFO]: Saved the model to tutorial_results/imputation/saits/20250316_T152030/SAITS.pypots


<h4>Train - MinMax Scaler</h4>

In [15]:
saits_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-16 15:23:01 [INFO]: Epoch 001 - training loss (MSE): 0.0360, validation MSE: 0.0082
2025-03-16 15:23:10 [INFO]: Epoch 002 - training loss (MSE): 0.0163, validation MSE: 0.0064
2025-03-16 15:23:19 [INFO]: Epoch 003 - training loss (MSE): 0.0124, validation MSE: 0.0056
2025-03-16 15:23:29 [INFO]: Epoch 004 - training loss (MSE): 0.0100, validation MSE: 0.0045
2025-03-16 15:23:38 [INFO]: Epoch 005 - training loss (MSE): 0.0083, validation MSE: 0.0039
2025-03-16 15:23:47 [INFO]: Epoch 006 - training loss (MSE): 0.0071, validation MSE: 0.0035
2025-03-16 15:23:57 [INFO]: Epoch 007 - training loss (MSE): 0.0064, validation MSE: 0.0036
2025-03-16 15:24:06 [INFO]: Epoch 008 - training loss (MSE): 0.0060, validation MSE: 0.0033
2025-03-16 15:24:16 [INFO]: Epoch 009 - training loss (MSE): 0.0057, validation MSE: 0.0032
2025-03-16 15:24:25 [INFO]: Epoch 010 - training loss (MSE): 0.0054, validation MSE: 0.0031
2025-03-16 15:24:25 [INFO]: Finished training. The best model is from epoch#10.


<h4> Load - Standard Scaler </h4>

In [22]:
saits_standard.load("tutorial_results/imputation/saits/standard/SAITS.pypots")

2025-03-16 17:47:39 [INFO]: Model loaded successfully from tutorial_results/imputation/saits/standard/SAITS.pypots


<h4> Load - Min/Max Scaler </h4>

In [23]:
saits_minmax.load("tutorial_results/imputation/saits/minmax/SAITS.pypots")

2025-03-16 17:47:41 [INFO]: Model loaded successfully from tutorial_results/imputation/saits/minmax/SAITS.pypots


### BRITS

<h4> Train - Standard Scaler </h4>

In [21]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-16 15:30:49 [INFO]: Epoch 001 - training loss (MSE): 0.9485, validation MSE: 6.7896
2025-03-16 15:31:26 [INFO]: Epoch 002 - training loss (MSE): 0.7392, validation MSE: 6.7438
2025-03-16 15:32:02 [INFO]: Epoch 003 - training loss (MSE): 0.6869, validation MSE: 6.7314
2025-03-16 15:32:39 [INFO]: Epoch 004 - training loss (MSE): 0.6619, validation MSE: 6.7251
2025-03-16 15:33:16 [INFO]: Epoch 005 - training loss (MSE): 0.6458, validation MSE: 6.7235
2025-03-16 15:33:52 [INFO]: Epoch 006 - training loss (MSE): 0.6336, validation MSE: 6.7230
2025-03-16 15:34:28 [INFO]: Epoch 007 - training loss (MSE): 0.6241, validation MSE: 6.7243
2025-03-16 15:35:05 [INFO]: Epoch 008 - training loss (MSE): 0.6159, validation MSE: 6.7273
2025-03-16 15:35:42 [INFO]: Epoch 009 - training loss (MSE): 0.6090, validation MSE: 6.7300
2025-03-16 15:35:42 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-03-16 15:35:42 [INFO]: Finished training. The best model is from epoc

<h4> Train - MinMax Scaler</h4>

In [22]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-16 15:38:39 [INFO]: Epoch 001 - training loss (MSE): 0.1627, validation MSE: 0.0075
2025-03-16 15:39:15 [INFO]: Epoch 002 - training loss (MSE): 0.0964, validation MSE: 0.0052
2025-03-16 15:39:52 [INFO]: Epoch 003 - training loss (MSE): 0.0817, validation MSE: 0.0044
2025-03-16 15:40:29 [INFO]: Epoch 004 - training loss (MSE): 0.0754, validation MSE: 0.0039
2025-03-16 15:41:05 [INFO]: Epoch 005 - training loss (MSE): 0.0711, validation MSE: 0.0036
2025-03-16 15:41:42 [INFO]: Epoch 006 - training loss (MSE): 0.0675, validation MSE: 0.0031
2025-03-16 15:42:19 [INFO]: Epoch 007 - training loss (MSE): 0.0647, validation MSE: 0.0029
2025-03-16 15:42:55 [INFO]: Epoch 008 - training loss (MSE): 0.0630, validation MSE: 0.0028
2025-03-16 15:43:32 [INFO]: Epoch 009 - training loss (MSE): 0.0615, validation MSE: 0.0028
2025-03-16 15:44:09 [INFO]: Epoch 010 - training loss (MSE): 0.0603, validation MSE: 0.0027
2025-03-16 15:44:09 [INFO]: Finished training. The best model is from epoch#10.


<h4>Load - Standard Scaler</h4>

In [24]:
brits_standard.load("tutorial_results/imputation/brits/standard/BRITS.pypots")

2025-03-16 17:47:49 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/standard/BRITS.pypots


<h4>Load - Min/Max Scaler</h4>

In [25]:
brits_minmax.load("tutorial_results/imputation/brits/minmax/BRITS.pypots")

2025-03-16 17:47:52 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/minmax/BRITS.pypots


### USGAN

<h4>Train - Standard Scaler </h4>

In [25]:
us_gan_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-16 15:48:06 [INFO]: Epoch 001 - generator training loss: 0.4295, discriminator training loss: 0.1932, validation loss: 6.7777
2025-03-16 15:49:33 [INFO]: Epoch 002 - generator training loss: 0.3657, discriminator training loss: 0.0552, validation loss: 6.7244
2025-03-16 15:51:00 [INFO]: Epoch 003 - generator training loss: 0.3306, discriminator training loss: 0.0364, validation loss: 6.7009
2025-03-16 15:52:27 [INFO]: Epoch 004 - generator training loss: 0.3129, discriminator training loss: 0.0304, validation loss: 6.6854
2025-03-16 15:53:54 [INFO]: Epoch 005 - generator training loss: 0.3034, discriminator training loss: 0.0283, validation loss: 6.6831
2025-03-16 15:55:22 [INFO]: Epoch 006 - generator training loss: 0.2971, discriminator training loss: 0.0274, validation loss: 6.6730
2025-03-16 15:56:49 [INFO]: Epoch 007 - generator training loss: 0.2880, discriminator training loss: 0.0258, validation loss: 6.6687
2025-03-16 15:58:16 [INFO]: Epoch 008 - generator training los

<h4> Train - MinMax Scaler</h4>

In [26]:
us_gan_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-16 16:02:58 [INFO]: Epoch 001 - generator training loss: -0.0626, discriminator training loss: 0.1841, validation loss: 0.0167
2025-03-16 16:04:25 [INFO]: Epoch 002 - generator training loss: -0.0035, discriminator training loss: 0.0544, validation loss: 0.0146
2025-03-16 16:05:51 [INFO]: Epoch 003 - generator training loss: -0.0007, discriminator training loss: 0.0373, validation loss: 0.0087
2025-03-16 16:07:18 [INFO]: Epoch 004 - generator training loss: -0.0050, discriminator training loss: 0.0321, validation loss: 0.0072
2025-03-16 16:08:45 [INFO]: Epoch 005 - generator training loss: -0.0054, discriminator training loss: 0.0295, validation loss: 0.0071
2025-03-16 16:10:12 [INFO]: Epoch 006 - generator training loss: -0.0053, discriminator training loss: 0.0276, validation loss: 0.0088
2025-03-16 16:11:39 [INFO]: Epoch 007 - generator training loss: -0.0043, discriminator training loss: 0.0264, validation loss: 0.0135
2025-03-16 16:13:06 [INFO]: Epoch 008 - generator train

<h4>Load - Standard Scaler</h4>

In [26]:
us_gan_standard.load("tutorial_results/imputation/us_gan/standard/USGAN.pypots")

2025-03-16 17:47:57 [INFO]: Model loaded successfully from tutorial_results/imputation/us_gan/standard/USGAN.pypots


<h4>Load - Min/Max Scaler</h4>

In [27]:
us_gan_minmax.load("tutorial_results/imputation/us_gan/minmax/USGAN.pypots")

2025-03-16 17:48:00 [INFO]: Model loaded successfully from tutorial_results/imputation/us_gan/minmax/USGAN.pypots


### GPVAE

<h4>Train - Standard Scaler</h4>

In [29]:
gp_vae_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-16 16:18:19 [INFO]: Epoch 001 - training loss (MSE): 25837.5240, validation MSE: 7.0216
2025-03-16 16:18:37 [INFO]: Epoch 002 - training loss (MSE): 22871.1035, validation MSE: 7.0068
2025-03-16 16:18:56 [INFO]: Epoch 003 - training loss (MSE): 22841.5796, validation MSE: 6.9927
2025-03-16 16:19:14 [INFO]: Epoch 004 - training loss (MSE): 22831.7891, validation MSE: 7.0024
2025-03-16 16:19:33 [INFO]: Epoch 005 - training loss (MSE): 22826.3858, validation MSE: 6.9474
2025-03-16 16:19:52 [INFO]: Epoch 006 - training loss (MSE): 22825.3808, validation MSE: 6.9542
2025-03-16 16:20:10 [INFO]: Epoch 007 - training loss (MSE): 22820.5412, validation MSE: 6.9309
2025-03-16 16:20:29 [INFO]: Epoch 008 - training loss (MSE): 22809.2307, validation MSE: 6.9157
2025-03-16 16:20:47 [INFO]: Epoch 009 - training loss (MSE): 22808.1378, validation MSE: 6.9200
2025-03-16 16:21:06 [INFO]: Epoch 010 - training loss (MSE): 22808.2470, validation MSE: 6.9055
2025-03-16 16:21:06 [INFO]: Finished tra

<h4>Train - MinMax Scaler</h4>

In [30]:
gp_vae_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-16 16:21:35 [INFO]: Epoch 001 - training loss (MSE): 24566.1815, validation MSE: 0.0213
2025-03-16 16:21:54 [INFO]: Epoch 002 - training loss (MSE): 22734.5193, validation MSE: 0.0108
2025-03-16 16:22:14 [INFO]: Epoch 003 - training loss (MSE): 22714.5095, validation MSE: 0.0109
2025-03-16 16:22:33 [INFO]: Epoch 004 - training loss (MSE): 22711.1865, validation MSE: 0.0109
2025-03-16 16:22:52 [INFO]: Epoch 005 - training loss (MSE): 22710.2332, validation MSE: 0.0108
2025-03-16 16:23:12 [INFO]: Epoch 006 - training loss (MSE): 22709.8284, validation MSE: 0.0109
2025-03-16 16:23:31 [INFO]: Epoch 007 - training loss (MSE): 22709.6122, validation MSE: 0.0108
2025-03-16 16:23:50 [INFO]: Epoch 008 - training loss (MSE): 22709.4575, validation MSE: 0.0109
2025-03-16 16:23:50 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-03-16 16:23:50 [INFO]: Finished training. The best model is from epoch#5.
2025-03-16 16:23:50 [INFO]: Saved the model to tutorial

<h4>Load - Standard Scaler</h4>

In [28]:
gp_vae_standard.load("tutorial_results/imputation/gp_vae/standard/GPVAE.pypots")

2025-03-16 17:48:04 [INFO]: Model loaded successfully from tutorial_results/imputation/gp_vae/standard/GPVAE.pypots


<h4>Load - MinMax Scaler</h4>

In [29]:
gp_vae_minmax.load("tutorial_results/imputation/gp_vae/minmax/GPVAE.pypots")

2025-03-16 17:48:06 [INFO]: Model loaded successfully from tutorial_results/imputation/gp_vae/minmax/GPVAE.pypots


### MRNN

<h4>Train - Standard Scaler</h4>

In [33]:
mrnn_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-16 16:32:29 [INFO]: Epoch 001 - training loss (MSE): 0.7716, validation MSE: 7.3092
2025-03-16 16:37:12 [INFO]: Epoch 002 - training loss (MSE): 0.5358, validation MSE: 7.2663
2025-03-16 16:41:56 [INFO]: Epoch 003 - training loss (MSE): 0.4939, validation MSE: 7.2537
2025-03-16 16:46:40 [INFO]: Epoch 004 - training loss (MSE): 0.4747, validation MSE: 7.2493
2025-03-16 16:51:23 [INFO]: Epoch 005 - training loss (MSE): 0.4629, validation MSE: 7.2440
2025-03-16 16:56:07 [INFO]: Epoch 006 - training loss (MSE): 0.4545, validation MSE: 7.2427
2025-03-16 17:00:51 [INFO]: Epoch 007 - training loss (MSE): 0.4412, validation MSE: 7.2428
2025-03-16 17:05:34 [INFO]: Epoch 008 - training loss (MSE): 0.4364, validation MSE: 7.2450
2025-03-16 17:10:18 [INFO]: Epoch 009 - training loss (MSE): 0.4295, validation MSE: 7.2484
2025-03-16 17:10:18 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-03-16 17:10:18 [INFO]: Finished training. The best model is from epoc

<h4>Train - MinMax Scaler</h4>

In [34]:
mrnn_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-16 17:24:52 [INFO]: Epoch 001 - training loss (MSE): 0.1931, validation MSE: 0.0290
2025-03-16 17:29:34 [INFO]: Epoch 002 - training loss (MSE): 0.0689, validation MSE: 0.0345
2025-03-16 17:34:16 [INFO]: Epoch 003 - training loss (MSE): 0.0532, validation MSE: 0.0433
2025-03-16 17:39:03 [INFO]: Epoch 004 - training loss (MSE): 0.0447, validation MSE: 0.0487
2025-03-16 17:39:03 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-03-16 17:39:03 [INFO]: Finished training. The best model is from epoch#1.
2025-03-16 17:39:03 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250316_T162708/MRNN.pypots


<h4>Load - Standard Scaler</h4>

In [30]:
mrnn_standard.load("tutorial_results/imputation/mrnn/standard/MRNN.pypots")

2025-03-16 17:48:10 [INFO]: Model loaded successfully from tutorial_results/imputation/mrnn/standard/MRNN.pypots


<h4>Load - MinMax Scaler</h4>

In [31]:
mrnn_minmax.load("tutorial_results/imputation/mrnn/minmax/MRNN.pypots")

2025-03-16 17:48:12 [INFO]: Model loaded successfully from tutorial_results/imputation/mrnn/minmax/MRNN.pypots


# Testing stage

### SAITS

#### Standard Scaler

In [32]:
saits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, saits_standard)
saits_imputation_standard = toolkits.pre_reshape(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.desnormalization(saits_imputation_standard, scaler_standard)
saits_imputation_standard = toolkits.reshape_variable(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.reshape_variable(saits_imputation_standard_ori)

#### MinMax Scaler

In [33]:
saits_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, saits_minmax)
saits_imputation_minmax = toolkits.pre_reshape(saits_imputation_minmax)
saits_imputation_minmax_ori = toolkits.desnormalization(saits_imputation_minmax, scaler_minmax)
saits_imputation_minmax = toolkits.reshape_variable(saits_imputation_minmax)
saits_imputation_minmax_ori = toolkits.reshape_variable(saits_imputation_minmax_ori)

### BRITS

#### Standard Scaler

In [34]:
brits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, brits_standard)
brits_imputation_standard = toolkits.pre_reshape(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.desnormalization(brits_imputation_standard, scaler_standard)
brits_imputation_standard = toolkits.reshape_variable(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.reshape_variable(brits_imputation_standard_ori)

#### MinMax Scaler

In [35]:
brits_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, brits_minmax)
brits_imputation_minmax = toolkits.pre_reshape(brits_imputation_minmax)
brits_imputation_minmax_ori = toolkits.desnormalization(brits_imputation_minmax, scaler_minmax) 
brits_imputation_minmax = toolkits.reshape_variable(brits_imputation_minmax)
brits_imputation_minmax_ori = toolkits.reshape_variable(brits_imputation_minmax_ori)  

### USGAN

#### Standard Scaler

In [36]:
usgan_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, us_gan_standard)
usgan_imputation_standard = toolkits.pre_reshape(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.desnormalization(usgan_imputation_standard, scaler_standard)
usgan_imputation_standard = toolkits.reshape_variable(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.reshape_variable(usgan_imputation_standard_ori)

#### MinMax Scaler

In [37]:
usgan_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, us_gan_minmax)
usgan_imputation_minmax = toolkits.pre_reshape(usgan_imputation_minmax)
usgan_imputation_minmax_ori = toolkits.desnormalization(usgan_imputation_minmax, scaler_minmax)
usgan_imputation_minmax = toolkits.reshape_variable(usgan_imputation_minmax)
usgan_imputation_minmax_ori = toolkits.reshape_variable(usgan_imputation_minmax_ori)

### GPVAE

#### Standard Scaler

In [38]:
gpvae_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, gp_vae_standard)
gpvae_imputation_standard = toolkits.pre_reshape(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.desnormalization(gpvae_imputation_standard, scaler_standard)
gpvae_imputation_standard = toolkits.reshape_variable(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.reshape_variable(gpvae_imputation_standard_ori) 

#### MinMax Scaler

In [39]:
gpvae_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, gp_vae_minmax)
gpvae_imputation_minmax = toolkits.pre_reshape(gpvae_imputation_minmax)
gpvae_imputation_minmax_ori = toolkits.desnormalization(gpvae_imputation_minmax, scaler_minmax)
gpvae_imputation_minmax = toolkits.reshape_variable(gpvae_imputation_minmax)
gpvae_imputation_minmax_ori = toolkits.reshape_variable(gpvae_imputation_minmax_ori) 

### MRNN

#### Standard Scaler

In [40]:
mrnn_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, mrnn_standard)
mrnn_imputation_standard = toolkits.pre_reshape(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.desnormalization(mrnn_imputation_standard, scaler_standard)
mrnn_imputation_standard = toolkits.reshape_variable(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.reshape_variable(mrnn_imputation_standard_ori) 

#### MinMax Scaler

In [41]:
mrnn_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, mrnn_minmax)
mrnn_imputation_minmax = toolkits.pre_reshape(mrnn_imputation_minmax)
mrnn_imputation_minmax_ori = toolkits.desnormalization(mrnn_imputation_minmax, scaler_minmax)
mrnn_imputation_minmax = toolkits.reshape_variable(mrnn_imputation_minmax)
mrnn_imputation_minmax_ori = toolkits.reshape_variable(mrnn_imputation_minmax_ori) 

# Calculate mean absolute error

### SAITS

#### Standard scaler (C/Normalização)

In [42]:
testing_mae_saits_variables_standard = toolkits.calculate_mae(saits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

#### Standard scaler (S/Normalização)

In [43]:
testing_mae_saits_variables_standard_ori = toolkits.calculate_mae(saits_imputation_standard_ori, test_X_ori_variable_standard_des, indicating_mask_variable_standard_des)

#### MinMax scaler (C/Normalização)

In [44]:
testing_mae_saits_variables_minmax = toolkits.calculate_mae(saits_imputation_minmax, test_X_ori_variable_minmax_norm, indicating_mask_variable_minmax_norm)

#### MinMax scaler (S/Normalização)

In [45]:
testing_mae_saits_variables_minmax_ori = toolkits.calculate_mae(saits_imputation_minmax_ori, test_X_ori_variable_minmax_des, indicating_mask_variable_minmax_des)

### BRITS

#### Standard Scaler (C/Normalização)

In [ ]:
testing_mae_brits_variables_standard = toolkits.calculate_mae(brits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

#### Standard Scaler (S/Normalização)

In [48]:
testing_mae_brits_variables_standard_ori = toolkits.calculate_mae(brits_imputation_standard_ori, test_X_ori_variable_standard_des, indicating_mask_variable_standard_des)

#### MinMax Scaler (C/Normalização)

In [49]:
testing_mae_brits_variables_minmax = toolkits.calculate_mae(brits_imputation_minmax, test_X_ori_variable_minmax_norm, indicating_mask_variable_minmax_norm)

#### MinMax Scaler (S/Normalização)

In [51]:
testing_mae_brits_variables_minmax_ori = toolkits.calculate_mae(brits_imputation_minmax_ori, test_X_ori_variable_minmax_des, indicating_mask_variable_minmax_des)

### USGAN

#### Standard Scaler (C/Normalização)

In [52]:
testing_mae_usgan_variables_standard = toolkits.calculate_mae(usgan_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

#### Standard Scaler (S/Normalização)

In [53]:
testing_mae_usgan_variables_standard_ori = toolkits.calculate_mae(usgan_imputation_standard_ori, test_X_ori_variable_standard_des, indicating_mask_variable_standard_des)

#### MinMax Scaler (C/Normalização)

In [54]:
testing_mae_usgan_variables_minmax = toolkits.calculate_mae(usgan_imputation_minmax, test_X_ori_variable_minmax_norm, indicating_mask_variable_minmax_norm)

#### MinMax Scaler (S/Normalização)

In [55]:
testing_mae_usgan_variables_minmax_ori = toolkits.calculate_mae(usgan_imputation_minmax_ori, test_X_ori_variable_minmax_des, indicating_mask_variable_minmax_des)

### GPVAE

#### Standard Scaler (C/Normalização)

In [56]:
testing_mae_gpvae_variables_standard = toolkits.calculate_mae(gpvae_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

#### Standard Scaler (S/Normalização)

In [57]:
testing_mae_gpvae_variables_standard_ori = toolkits.calculate_mae(gpvae_imputation_standard_ori, test_X_ori_variable_standard_des, indicating_mask_variable_standard_des)

#### MinMax Scaler (C/Normalização)

In [58]:
testing_mae_gpvae_variables_minmax = toolkits.calculate_mae(gpvae_imputation_minmax, test_X_ori_variable_minmax_norm, indicating_mask_variable_minmax_norm)

#### MinMax Scaler (S/Normalização)

In [59]:
testing_mae_gpvae_variables_minmax_ori = toolkits.calculate_mae(gpvae_imputation_minmax_ori, test_X_ori_variable_minmax_des, indicating_mask_variable_minmax_des)

### MRNN

#### Standard Scaler (C/Normalização)

In [60]:
testing_mae_mrnn_variables_standard = toolkits.calculate_mae(mrnn_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

#### Standard Scaler (S/Normalização)

In [61]:
testing_mae_mrnn_variables_standard_ori = toolkits.calculate_mae(mrnn_imputation_standard_ori, test_X_ori_variable_standard_des, indicating_mask_variable_standard_des)

#### MinMax Scaler (C/Normalização)

In [62]:
testing_mae_mrnn_variables_minmax = toolkits.calculate_mae(mrnn_imputation_minmax, test_X_ori_variable_minmax_norm, indicating_mask_variable_minmax_norm)

#### MinMax Scaler (S/Normalização)

In [63]:
testing_mae_mrnn_variables_minmax_ori = toolkits.calculate_mae(mrnn_imputation_minmax_ori, test_X_ori_variable_minmax_des, indicating_mask_variable_minmax_des)

# Results mean absolute error

In [64]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

In [65]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

#### SAITS

<h4>Standard Scaler (C/Normalização)</h4>

In [66]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 0.35549184664706085
ALT : 0.34616397235246166
AST : 0.283203929613111
Albumin : 0.5865378684716107
BUN : 0.23549456829210566
Bilirubin : 0.29213390018033225
Cholesterol : 0.8619142944054452
Creatinine : 0.2272360108358054
DiasABP : 0.25696352027850733
FiO2 : 0.4843457493583031
GCS : 0.27782594851598286
Glucose : 0.5574566141755575
HCO3 : 0.42235495223366226
HCT : 0.5478476366595461
HR : 0.2786515469045242
K : 0.6173738181988982
Lactate : 0.3602081488093162
MAP : 0.276148040474521
MechVent : 0.016131509283529688
Mg : 0.5428868319523208
NIDiasABP : 0.23805346595387422
NIMAP : 0.17731126967659114
NISysABP : 0.3059993623684565
Na : 0.5094466185029012
PaCO2 : 0.5448751678244528
PaO2 : 0.5044175503727509
Platelets : 0.37910753219049526
RespRate : 0.4975549864022565
SaO2 : 0.395203639666887
SysABP : 0.2876315375696396
Temp : 0.3654739421516718
TroponinI : 0.5742820858879293
TroponinT : 0.562502744207675
Urine : 0.4555801478350003
WBC : 0.15

In [67]:
df_saits_mae_standard = toolkits.create_table(testing_mae_saits_variables_standard, subgroups, variables)

In [68]:
df_saits_mae_standard

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.355492,0.398791,0.388989,0.000000,0.331751,0.740879,0.222358,0.195952,0.723086,0.505640,0.296971,0.216395,0.284060,0.236059,0.184009,0.347203,0.266653
1,ALT,0.346164,0.267721,0.253539,0.000000,0.280160,0.329866,0.296968,0.351267,0.311489,0.272558,0.325780,0.238700,0.251692,0.305938,0.274798,0.318639,0.823624
2,AST,0.283204,0.257204,0.204749,0.000000,0.206225,0.243144,0.340113,0.273045,0.305834,0.306141,0.244520,0.699315,0.218977,0.318423,0.207619,0.237748,0.209382
3,Albumin,0.586538,0.609286,0.621456,0.000000,0.536556,0.658661,0.490084,0.717311,0.708582,0.754085,0.776171,1.172093,0.840292,0.814925,0.467807,0.670209,0.729714
4,BUN,0.235495,0.253363,0.228878,0.000000,0.242391,0.244288,0.213308,0.222898,0.294829,0.209661,0.257893,0.200723,0.219590,0.235670,0.210442,0.309354,0.251667
5,Bilirubin,0.292134,0.301597,0.249420,0.000000,0.231827,0.336409,0.200403,0.318250,0.280385,0.267887,0.312005,0.954099,0.392959,0.242916,0.335206,0.310083,0.216768
6,Cholesterol,0.861914,1.451439,0.918617,0.000000,0.736495,1.183795,1.002457,0.000000,0.906296,0.725019,0.618523,0.085870,0.704775,1.232571,0.533449,0.347422,0.000000
7,Creatinine,0.227236,0.218419,0.210438,0.000000,0.200026,0.211177,0.193449,0.183120,0.265242,0.217446,0.233475,0.177189,0.227362,0.206401,0.179911,0.192612,0.192658
8,DiasABP,0.256964,0.260941,0.240924,0.182851,0.271154,0.231552,0.299906,0.221175,0.250333,0.241564,0.250752,0.246645,0.246105,0.255920,0.251888,0.235695,0.250253
9,FiO2,0.484346,0.452276,0.517671,0.000000,0.491061,0.501327,0.482516,0.467761,0.507122,0.438264,0.460432,0.476691,0.439314,0.414027,0.507958,0.501128,0.573521


In [69]:
latex_code = df_saits_mae_standard.to_latex(index=False)

with open("tabela.tex", "w") as f:
    f.write(latex_code)

In [70]:
print(latex_code)

\begin{tabular}{lrrrrrrrrrrrrrrrrr}
\toprule
0 & General & Female & Male & Undefined Gender & +65 & -65 & ICUType 1 & ICUType 2 & ICUType 3 & ICUType 4 & Undefined classification & Low Weight & Normal Weight & Overweight & Obesity 1 & Obesity 2 & Obesity 3 \\
\midrule
ALP & 0.355492 & 0.398791 & 0.388989 & 0.000000 & 0.331751 & 0.740879 & 0.222358 & 0.195952 & 0.723086 & 0.505640 & 0.296971 & 0.216395 & 0.284060 & 0.236059 & 0.184009 & 0.347203 & 0.266653 \\
ALT & 0.346164 & 0.267721 & 0.253539 & 0.000000 & 0.280160 & 0.329866 & 0.296968 & 0.351267 & 0.311489 & 0.272558 & 0.325780 & 0.238700 & 0.251692 & 0.305938 & 0.274798 & 0.318639 & 0.823624 \\
AST & 0.283204 & 0.257204 & 0.204749 & 0.000000 & 0.206225 & 0.243144 & 0.340113 & 0.273045 & 0.305834 & 0.306141 & 0.244520 & 0.699315 & 0.218977 & 0.318423 & 0.207619 & 0.237748 & 0.209382 \\
Albumin & 0.586538 & 0.609286 & 0.621456 & 0.000000 & 0.536556 & 0.658661 & 0.490084 & 0.717311 & 0.708582 & 0.754085 & 0.776171 & 1.172093 & 0.84029

<h5>Minimum MAE value in each subgroup</h5>

In [71]:
toolkits.min_value_in_subgroup(df_saits_mae_standard, subgroups, variables)

General
MechVent
0.016131509283529688
--------------------
Female
MechVent
0.014552340041130598
--------------------
Male
MechVent
0.014512000309648741
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.012942020909290175
--------------------
-65
MechVent
0.01827778097688463
--------------------
ICUType 1
MechVent
0.012979781499681835
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.015398824789147038
--------------------
ICUType 4
MechVent
0.01676655673093299
--------------------
Undefined classification
MechVent
0.015602762801210616
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.013784697335756175
--------------------
Overweight
MechVent
0.014702130836156856
--------------------
Obesity 1
MechVent
0.014115385650734221
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [72]:
toolkits.max_value_in_subgroup(df_saits_mae_standard, subgroups, variables)

General
Cholesterol
0.8619142944054452
--------------------
Female
Cholesterol
1.451438736858023
--------------------
Male
Cholesterol
0.9186172160150028
--------------------
Undefined Gender
K
0.6481980955579346
--------------------
+65
TroponinI
0.912479961059112
--------------------
-65
Cholesterol
1.1837952493687076
--------------------
ICUType 1
TroponinI
1.4147481358654532
--------------------
ICUType 2
Albumin
0.7173112394561357
--------------------
ICUType 3
SaO2
1.5843908865861114
--------------------
ICUType 4
TroponinI
0.8606517177368178
--------------------
Undefined classification
Albumin
0.7761709655447225
--------------------
Low Weight
Albumin
1.1720931746200332
--------------------
Normal Weight
Albumin
0.8402915751610249
--------------------
Overweight
Cholesterol
1.2325708781179912
--------------------
Obesity 1
Ph
1.006255796754588
--------------------
Obesity 2
TroponinT
1.4516069413777122
--------------------
Obesity 3
TroponinT
1.03692539683769
------------------

<h4>Standard Scaler (S/Normalização)</h4>

In [73]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard_ori, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 46.05178056160585
ALT : 406.2112769981204
AST : 496.992055665294
Albumin : 0.3851788459064264
BUN : 5.380332453297799
Bilirubin : 1.6736765518848495
Cholesterol : 38.400731277463905
Creatinine : 0.3503998682295476
DiasABP : 3.286241163141612
FiO2 : 0.09104833207391178
GCS : 1.1115600990723302
Glucose : 35.67556060432917
HCO3 : 1.9973297300677588
HCT : 2.7491760875685074
HR : 4.954845594432205
K : 0.4188106400472619
Lactate : 0.9578556416874965
MAP : 4.378979286697319
MechVent : 0.01613150967851928
Mg : 0.2635250708242739
NIDiasABP : 3.5702379063267773
NIMAP : 2.6413458948607875
NISysABP : 6.786345809909647
Na : 2.6723842429156246
PaCO2 : 4.967970567426776
PaO2 : 43.05042321078405
Platelets : 40.793265397332874
RespRate : 2.670931693010336
SaO2 : 1.3594024396249635
SysABP : 6.740216718958554
Temp : 0.466368066859829
TroponinI : 6.362012470364254
TroponinT : 1.6937565297072215
Urine : 71.26128697044311
WBC : 6.241771727095261
Weight : 

In [74]:
df_saits_mae_standard_ori = toolkits.create_table(testing_mae_saits_variables_standard_ori, subgroups, variables)

In [75]:
df_saits_mae_standard_ori

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,46.051781,51.660965,50.391101,0.000000,42.976266,95.976280,28.805102,25.384391,93.671351,65.502523,38.470790,28.032644,36.798283,30.579934,23.837229,44.978069,34.543202
1,ALT,406.211277,314.161357,297.518824,0.000000,328.757487,387.086329,348.481409,412.199358,365.521484,319.837259,382.291076,280.105937,295.352176,359.007188,322.465492,373.911811,966.493927
2,AST,496.992056,451.364648,359.312336,0.000000,361.902289,426.691253,596.861957,479.165045,536.706173,537.244698,429.105660,1227.220856,384.279929,558.798516,364.348525,417.221308,367.442741
3,Albumin,0.385179,0.400118,0.408109,0.000000,0.352356,0.432542,0.321838,0.471058,0.465325,0.495207,0.509711,0.769713,0.551819,0.535161,0.307209,0.440125,0.479202
4,BUN,5.380332,5.788564,5.229164,0.000000,5.537895,5.581244,4.873432,5.092528,6.735940,4.790106,5.892070,4.585910,5.016969,5.384336,4.807965,7.067806,5.749823
5,Bilirubin,1.673677,1.727891,1.428961,0.000000,1.328168,1.927334,1.148139,1.823300,1.606364,1.534762,1.787523,5.466168,2.251319,1.391699,1.920443,1.776509,1.241894
6,Cholesterol,38.400731,64.665720,40.927007,0.000000,32.812929,52.741440,44.662291,0.000000,40.378078,32.301643,27.556951,3.825745,31.399719,54.914536,23.766652,15.478622,0.000000
7,Creatinine,0.350400,0.336804,0.324496,0.000000,0.308442,0.325636,0.298301,0.282373,0.409005,0.335304,0.360020,0.273227,0.350594,0.318273,0.277424,0.297009,0.297080
8,DiasABP,3.286241,3.337108,3.081119,2.338437,3.467723,2.961258,3.835419,2.828557,3.201445,3.089303,3.206805,3.154283,3.147369,3.272900,3.221336,3.014249,3.200419
9,FiO2,0.091048,0.085020,0.097313,0.000000,0.092311,0.094241,0.090704,0.087931,0.095330,0.082386,0.086553,0.089609,0.082583,0.077830,0.095487,0.094203,0.107812


<h5>Minimum MAE value in each subgroup</h5>

In [76]:
toolkits.min_value_in_subgroup(df_saits_mae_standard_ori, subgroups, variables)

General
MechVent
0.01613150967851928
--------------------
Female
MechVent
0.01455234083128562
--------------------
Male
MechVent
0.01451199920706472
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.012942019615087409
--------------------
-65
MechVent
0.01827777949621049
--------------------
ICUType 1
MechVent
0.01297977845445802
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.015398822799574146
--------------------
ICUType 4
MechVent
0.016766555871361
--------------------
Undefined classification
MechVent
0.015602763073054438
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.01378469708507042
--------------------
Overweight
MechVent
0.01470213082798736
--------------------
Obesity 1
MechVent
0.014115385917010151
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [77]:
toolkits.max_value_in_subgroup(df_saits_mae_standard_ori, subgroups, variables)

General
AST
496.992055665294
--------------------
Female
AST
451.3646475829517
--------------------
Male
AST
359.3123356241838
--------------------
Undefined Gender
Urine
65.78923797600842
--------------------
+65
AST
361.9022886474364
--------------------
-65
AST
426.6912528872452
--------------------
ICUType 1
AST
596.8619574737311
--------------------
ICUType 2
AST
479.1650450333097
--------------------
ICUType 3
AST
536.7061729221494
--------------------
ICUType 4
AST
537.2446980476282
--------------------
Undefined classification
AST
429.10566036324303
--------------------
Low Weight
AST
1227.220855712584
--------------------
Normal Weight
AST
384.2799291610597
--------------------
Overweight
AST
558.7985157739283
--------------------
Obesity 1
AST
364.3485252380189
--------------------
Obesity 2
AST
417.2213083901604
--------------------
Obesity 3
ALT
966.4939270016309
--------------------


<h4>MinMax Scaler (C/Normalização)</h4>

In [79]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_minmax, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 0.040219907134693526
ALT : 0.017229766448887466
AST : 0.015468583460460567
Albumin : 0.45373933730150834
BUN : 0.11917518228086253
Bilirubin : 0.04461143951106635
Cholesterol : 0.38712575286624884
Creatinine : 0.06330636563215755
DiasABP : 0.2222883897032941
FiO2 : 0.4212544503171473
GCS : 0.7013794357043793
Glucose : 0.08704999971751116
HCO3 : 0.38295172317766624
HCT : 0.45079366319242514
HR : 0.43347807748668743
K : 0.12318451615964963
Lactate : 0.08926922807197073
MAP : 0.2694956399200512
MechVent : 0.0
Mg : 0.06937552593284982
NIDiasABP : 0.34968573475266584
NIMAP : 0.34092245932783305
NISysABP : 0.45429145035202567
Na : 0.4986517947522833
PaCO2 : 0.4095278180110242
PaO2 : 0.29658351907452635
Platelets : 0.08119226622028226
RespRate : 0.19828358249685923
SaO2 : 0.9685085946654024
SysABP : 0.42148474257749563
Temp : 0.9335184893617962
TroponinI : 0.16517311730421255
TroponinT : 0.02879763797807984
Urine : 0.010554451556121645
WBC 

In [80]:
df_saits_mae_minmax = toolkits.create_table(testing_mae_saits_variables_minmax, subgroups, variables)

In [81]:
df_saits_mae_minmax

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.040220,0.041253,0.042669,0.000000,0.045370,0.058368,0.029260,0.018007,0.063298,0.040524,0.036990,0.128495,0.039369,0.032560,0.033538,0.019912,0.031346
1,ALT,0.017230,0.006941,0.016567,0.000000,0.012877,0.023744,0.018795,0.029335,0.023352,0.016529,0.017182,0.012010,0.009226,0.037875,0.014711,0.045587,0.099730
2,AST,0.015469,0.017081,0.006773,0.000000,0.006469,0.011712,0.018681,0.008078,0.013382,0.017210,0.007568,0.024316,0.004764,0.018236,0.010571,0.002253,0.001133
3,Albumin,0.453739,0.423997,0.450085,0.000000,0.428799,0.450342,0.544186,0.392027,0.419730,0.411677,0.404592,0.488372,0.380661,0.470012,0.461240,0.406977,0.390698
4,BUN,0.119175,0.115023,0.134813,0.000000,0.147150,0.102920,0.136926,0.092971,0.158645,0.091490,0.125755,0.143510,0.128078,0.122560,0.140897,0.149504,0.141484
5,Bilirubin,0.044611,0.034648,0.036118,0.000000,0.026320,0.053072,0.018808,0.037440,0.039132,0.045804,0.044419,0.133333,0.055639,0.025541,0.041761,0.026481,0.035695
6,Cholesterol,0.387126,0.473054,0.389912,0.000000,0.363024,0.238024,0.430447,0.000000,0.300150,0.356287,0.427146,0.377246,0.258982,0.533790,0.311377,0.407186,0.000000
7,Creatinine,0.063306,0.048783,0.072079,0.000000,0.057820,0.060337,0.079545,0.051091,0.086067,0.046976,0.065030,0.041558,0.068246,0.056569,0.057444,0.059394,0.076540
8,DiasABP,0.222288,0.221258,0.223552,0.229701,0.212173,0.235335,0.217357,0.208378,0.226277,0.231468,0.230916,0.218497,0.215240,0.216017,0.220501,0.222197,0.221234
9,FiO2,0.421254,0.407955,0.441474,0.000000,0.431625,0.422815,0.415464,0.454358,0.438086,0.399443,0.399171,0.374807,0.413956,0.412962,0.471615,0.444020,0.479529


<h5>Minimum MAE value in each subgroup</h5>

In [82]:
toolkits.min_value_in_subgroup(df_saits_mae_minmax, subgroups, variables)

General
MechVent
0.0
--------------------
Female
MechVent
0.0
--------------------
Male
MechVent
0.0
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.0
--------------------
-65
MechVent
0.0
--------------------
ICUType 1
MechVent
0.0
--------------------
ICUType 2
MechVent
0.0
--------------------
ICUType 3
MechVent
0.0
--------------------
ICUType 4
MechVent
0.0
--------------------
Undefined classification
MechVent
0.0
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.0
--------------------
Overweight
MechVent
0.0
--------------------
Obesity 1
MechVent
0.0
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
MechVent
0.0
--------------------


<h5> Maximum MAE value in each subgroup </h5>

In [83]:
toolkits.max_value_in_subgroup(df_saits_mae_minmax, subgroups, variables)

General
SaO2
0.9685085946654024
--------------------
Female
SaO2
0.9617415842380362
--------------------
Male
SaO2
0.9675699026353866
--------------------
Undefined Gender
GCS
0.9375000149009268
--------------------
+65
SaO2
0.9675195857723378
--------------------
-65
SaO2
0.9603608350471073
--------------------
ICUType 1
SaO2
0.964683557990219
--------------------
ICUType 2
SaO2
0.9694730503594138
--------------------
ICUType 3
Temp
0.9274873339532732
--------------------
ICUType 4
SaO2
0.9657142966512496
--------------------
Undefined classification
SaO2
0.9598484974015699
--------------------
Low Weight
SaO2
0.966071435383319
--------------------
Normal Weight
SaO2
0.9706250113936478
--------------------
Overweight
SaO2
0.9674803281393501
--------------------
Obesity 1
SaO2
0.9677868954470741
--------------------
Obesity 2
SaO2
0.962777776850541
--------------------
Obesity 3
SaO2
0.9603703794655621
--------------------


<h4>MinMax Scaler (S/Normalização)</h4>

In [84]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_minmax_ori, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 120.21354016661581
ALT : 292.51042278607537
AST : 566.9945693689813
Albumin : 2.9510791413217787
BUN : 25.788438117812007
Bilirubin : 2.7703703989979305
Cholesterol : 157.29999999999214
Creatinine : 1.4927400290198274
DiasABP : 59.684729736472974
FiO2 : 0.5427910102481897
GCS : 11.416553277655188
Glucose : 139.70664855678496
HCO3 : 22.998730468265872
HCT : 30.609207015520653
HR : 86.69561557043218
K : 4.13614858243916
Lactate : 2.767346140115878
MAP : 80.30970073528962
MechVent : 0.9999999999999996
Mg : 2.011890264982128
NIDiasABP : 58.44698860910438
NIMAP : 77.73032076927599
NISysABP : 119.4786514127118
Na : 138.88944723618073
PaCO2 : 40.95278192858044
PaO2 : 148.29175956176658
Platelets : 190.6867094356714
RespRate : 19.431791012219442
SaO2 : 96.8508583690985
SysABP : 120.12315150566705
Temp : 37.09088744652194
TroponinI : 8.210000050067492
TroponinT : 0.8710493570492538
Urine : 116.09896736077816
WBC : 12.26488773373189
Weight : 8

In [85]:
df_saits_mae_minmax_ori = toolkits.create_table(testing_mae_saits_variables_minmax_ori, subgroups, variables)

In [86]:
df_saits_mae_minmax_ori 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,120.213540,123.095891,127.046296,0.000000,134.583333,170.847619,89.636364,58.240000,184.601941,121.060607,111.202248,366.500000,117.838710,98.843750,101.571429,63.555556,95.454543
1,ALT,292.510423,118.436620,281.300001,0.000000,218.868130,402.716417,319.000002,497.315789,396.091955,280.655731,291.708863,204.200000,157.095240,641.805538,249.900001,772.285714,1688.333333
2,AST,566.994569,625.684209,250.518520,0.000000,239.448055,430.263396,683.920050,298.000001,491.054948,630.363636,279.442109,889.000068,177.375000,667.714272,388.750004,86.000001,45.250000
3,Albumin,2.951079,2.823188,2.935366,0.000000,2.843836,2.936471,3.340000,2.685714,2.804839,2.770213,2.739744,3.100000,2.636842,3.021053,2.983333,2.750000,2.680000
4,BUN,25.788438,24.924859,29.041096,0.000000,31.607143,22.407408,29.480620,20.337931,33.998246,20.029915,27.157143,30.850000,27.640288,26.492424,30.306667,32.096774,30.428572
5,Bilirubin,2.770370,2.151613,2.242935,0.000000,1.634444,3.295798,1.168000,2.325000,2.430097,2.844444,2.758427,8.280000,3.455172,1.586111,2.593333,1.644445,2.216667
6,Cholesterol,157.300000,185.999994,158.230769,0.000000,149.250000,107.500000,171.769229,0.000000,128.250000,147.000000,170.666665,154.000000,114.499992,206.285712,132.000000,163.999985,0.000000
7,Creatinine,1.492740,1.173219,1.685744,0.000000,1.372038,1.427404,1.850000,1.224000,1.993464,1.133466,1.530662,1.014286,1.601408,1.344526,1.363768,1.406667,1.783871
8,DiasABP,59.684730,59.403303,60.029592,61.708333,56.923331,63.246397,58.338428,55.887238,60.773501,62.190792,62.040018,58.649750,57.760640,57.972569,59.196723,59.659678,59.396970
9,FiO2,0.542791,0.532284,0.558764,0.000000,0.550984,0.544024,0.538217,0.568943,0.556088,0.525560,0.525345,0.506098,0.537025,0.536240,0.582576,0.560776,0.588828


<h5>Minimum MAE value in each subgroup</h5>

In [87]:
toolkits.min_value_in_subgroup(df_saits_mae_minmax_ori, subgroups, variables)

General
FiO2
0.5427910102481897
--------------------
Female
FiO2
0.5322841955806464
--------------------
Male
FiO2
0.558764148292676
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
FiO2
0.5509840135837525
--------------------
-65
FiO2
0.5440240574521423
--------------------
ICUType 1
FiO2
0.5382165582316663
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
TroponinT
0.4359321938985411
--------------------
ICUType 4
TroponinT
0.4575000104183762
--------------------
Undefined classification
FiO2
0.5253448272903851
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
FiO2
0.5370249722019451
--------------------
Overweight
FiO2
0.5362397804247246
--------------------
Obesity 1
FiO2
0.5825757534816982
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [88]:
toolkits.max_value_in_subgroup(df_saits_mae_minmax_ori, subgroups, variables)

General
AST
566.9945693689813
--------------------
Female
AST
625.6842090706995
--------------------
Male
ALT
281.3000011026836
--------------------
Undefined Gender
Urine
139.99999999986
--------------------
+65
AST
239.44805549026773
--------------------
-65
AST
430.2633963482682
--------------------
ICUType 1
AST
683.9200496291841
--------------------
ICUType 2
ALT
497.31578862037054
--------------------
ICUType 3
AST
491.05494833516013
--------------------
ICUType 4
AST
630.3636355139878
--------------------
Undefined classification
ALT
291.7088629807061
--------------------
Low Weight
AST
889.0000677106542
--------------------
Normal Weight
AST
177.37499982118052
--------------------
Overweight
AST
667.7142724082266
--------------------
Obesity 1
AST
388.75000405309646
--------------------
Obesity 2
ALT
772.2857140131256
--------------------
Obesity 3
ALT
1688.3333333327705
--------------------


#### BRITS

<h4> Standard Scaler (C/Normalização)</h4>

In [89]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_standard, subgroups, variables)

BRITS - MAE
************
General
-------------
ALP : 0.4591248631105258
ALT : 0.2098298479270722
AST : 0.27798859929904113
Albumin : 0.6778925772529109
BUN : 0.39521960833661646
Bilirubin : 0.3833806392125974
Cholesterol : 0.9463970561439892
Creatinine : 0.3444718062995419
DiasABP : 0.20806233248448402
FiO2 : 0.34404522156046397
GCS : 0.21322463397612945
Glucose : 0.5644091100017328
HCO3 : 0.5047606820899914
HCT : 0.6338702223060172
HR : 0.2431173715923441
K : 0.6534995938478387
Lactate : 0.4167839664888235
MAP : 0.1564062923159312
MechVent : 0.007398844359593802
Mg : 0.5955810654854342
NIDiasABP : 0.15725503986364742
NIMAP : 0.09992748699932849
NISysABP : 0.2543230216923866
Na : 0.645442078485974
PaCO2 : 1.1895743136362507
PaO2 : 0.4776378612780617
Platelets : 0.6445916196510731
RespRate : 0.475295251812804
SaO2 : 0.348066606252948
SysABP : 0.25398352012962805
Temp : 0.240561647460938
TroponinI : 0.667214116433886
TroponinT : 0.29949287221387894
Urine : 0.3333883323992429
WBC : 0.1101

In [90]:
df_brits_mae_standard = toolkits.create_table(testing_mae_brits_variables_standard, subgroups, variables)

In [91]:
df_brits_mae_standard 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.459125,0.472351,0.521596,0.000000,0.613928,1.063688,0.214294,0.195314,1.131936,0.488011,0.397904,2.585976,0.496290,0.277286,0.269754,0.188530,0.169188
1,ALT,0.209830,0.101859,0.179522,0.000000,0.166618,0.273544,0.258180,0.337654,0.299586,0.196966,0.191726,0.177997,0.106276,0.432766,0.151008,0.527028,0.959179
2,AST,0.277989,0.302443,0.121857,0.000000,0.112314,0.180192,0.331781,0.134712,0.237710,0.282366,0.131539,0.433593,0.101432,0.257115,0.160427,0.067755,0.040300
3,Albumin,0.677893,0.665511,0.661255,0.000000,0.583676,0.709475,0.779543,0.885213,0.782780,0.771453,0.838277,0.777632,0.885481,0.837840,0.667693,0.733846,0.642471
4,BUN,0.395220,0.411375,0.473140,0.000000,0.464619,0.389517,0.456941,0.268321,0.610310,0.333333,0.455864,0.381064,0.478768,0.413387,0.459124,0.616949,0.483605
5,Bilirubin,0.383381,0.283809,0.268491,0.000000,0.206176,0.458713,0.092110,0.272721,0.324494,0.378855,0.387339,1.406046,0.509900,0.184905,0.392315,0.234792,0.175882
6,Cholesterol,0.946397,1.316313,0.887540,0.000000,0.682374,1.073912,0.901075,0.000000,0.675238,0.792898,0.535888,0.047612,0.804394,1.407452,0.599187,0.134502,0.000000
7,Creatinine,0.344472,0.246912,0.408667,0.000000,0.273083,0.358847,0.419311,0.248838,0.555938,0.236049,0.416430,0.205580,0.426131,0.277015,0.275718,0.259017,0.412865
8,DiasABP,0.208062,0.209535,0.202643,0.229810,0.203604,0.192454,0.236780,0.163427,0.237014,0.199740,0.229002,0.273534,0.184647,0.197253,0.202743,0.269852,0.166893
9,FiO2,0.344045,0.342385,0.388816,0.000000,0.354816,0.350545,0.374378,0.381264,0.376814,0.304279,0.347786,0.415828,0.341317,0.319320,0.347271,0.341069,0.464918


<h5>Minimum MAE value in each subgroup</h5>

In [92]:
toolkits.min_value_in_subgroup(df_brits_mae_standard, subgroups, variables)

General
MechVent
0.007398844359593802
--------------------
Female
MechVent
0.006747703165370343
--------------------
Male
MechVent
0.007949415685690596
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.006393708231510704
--------------------
-65
MechVent
0.006916098969701905
--------------------
ICUType 1
MechVent
0.008637714899469963
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.00760112133350753
--------------------
ICUType 4
MechVent
0.007131627306736024
--------------------
Undefined classification
MechVent
0.007929447085111131
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.006744162132749256
--------------------
Overweight
MechVent
0.005137604424881064
--------------------
Obesity 1
MechVent
0.005072495616452731
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [93]:
toolkits.max_value_in_subgroup(df_brits_mae_standard, subgroups, variables) 

General
PaCO2
1.1895743136362507
--------------------
Female
Cholesterol
1.316313407512585
--------------------
Male
SaO2
0.9631436680048339
--------------------
Undefined Gender
K
0.6736702746131231
--------------------
+65
SaO2
1.119685603436657
--------------------
-65
Cholesterol
1.073912175651407
--------------------
ICUType 1
TroponinI
1.3357495404853446
--------------------
ICUType 2
SaO2
1.0038150117625755
--------------------
ICUType 3
SaO2
1.5844865910892845
--------------------
ICUType 4
TroponinI
0.8218961397479049
--------------------
Undefined classification
Albumin
0.838276900298598
--------------------
Low Weight
ALP
2.5859758306247667
--------------------
Normal Weight
SaO2
2.130322246245731
--------------------
Overweight
Cholesterol
1.4074519849451501
--------------------
Obesity 1
Ph
0.9219915917887068
--------------------
Obesity 2
TroponinT
1.5021143882386014
--------------------
Obesity 3
TroponinT
1.1786538213799647
--------------------


<h4> Standard Scaler (S/Normalização)</h4>

In [94]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_standard_ori, subgroups, variables)

BRITS - MAE
************
General
-------------
ALP : 59.47679960727661
ALT : 246.22796522577477
AST : 487.83971088346965
Albumin : 0.44517139016295193
BUN : 9.029562419924192
Bilirubin : 2.1964420588243456
Cholesterol : 42.16467819213657
Creatinine : 0.5311784632312044
DiasABP : 2.6608562630997246
FiO2 : 0.06467434470693546
GCS : 0.8530952328724266
Glucose : 36.120499751403045
HCO3 : 2.3870289882427516
HCT : 3.180849491741162
HR : 4.322994241035456
K : 0.44331744076969337
Lactate : 1.1083005125962981
MAP : 2.4801911598377284
MechVent : 0.007398843992015586
Mg : 0.2891036038573192
NIDiasABP : 2.358452911676838
NIMAP : 1.4885859312337142
NISysABP : 5.640286130216485
Na : 3.3857702897421635
PaCO2 : 10.846099752981868
PaO2 : 40.76486252641003
Platelets : 69.36026018455121
RespRate : 2.5514389378585136
SaO2 : 1.1972626616514777
SysABP : 5.951725510414872
Temp : 0.30697198854512336
TroponinI : 7.391532210111248
TroponinT : 0.9018053834673745
Urine : 52.14819328993126
WBC : 4.548634074852356


In [95]:
df_brits_mae_standard_ori = toolkits.create_table(testing_mae_brits_variables_standard_ori, subgroups, variables)

In [96]:
df_brits_mae_standard_ori

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,59.476800,61.190113,67.569603,0.000000,79.530553,137.794253,27.760474,25.301766,146.635288,63.218761,51.545977,334.997253,64.291362,35.920715,34.945033,24.422891,21.917294
1,ALT,246.227965,119.528435,210.662644,0.000000,195.520338,320.993806,302.965329,396.225688,351.553458,231.133214,224.983331,208.873646,124.711506,507.836184,177.203073,618.448957,1125.562805
2,AST,487.839711,530.755241,213.846511,0.000000,197.098653,316.217424,582.239977,236.404832,417.154657,495.522346,230.836143,760.908161,178.002037,451.209064,281.532368,118.902110,70.722878
3,Albumin,0.445171,0.437040,0.434245,0.000000,0.383299,0.465912,0.511925,0.581319,0.514051,0.506612,0.550496,0.510670,0.581495,0.550209,0.438474,0.481916,0.421910
4,BUN,9.029562,9.398659,10.809797,0.000000,10.615120,8.899264,10.439719,6.130315,13.943719,7.615652,10.415096,8.706156,10.938398,9.444630,10.489574,14.095400,11.048900
5,Bilirubin,2.196442,1.625982,1.538225,0.000000,1.181210,2.628034,0.527712,1.562459,1.859073,2.170516,2.219119,8.055434,2.921292,1.059348,2.247626,1.345154,1.007650
6,Cholesterol,42.164678,58.645502,39.542426,0.000000,30.401688,47.845840,40.145447,0.000000,30.083759,35.325846,23.875329,2.121262,35.838058,62.705985,26.695491,5.992462,0.000000
7,Creatinine,0.531178,0.380740,0.630169,0.000000,0.421096,0.553345,0.646581,0.383710,0.857260,0.363990,0.642139,0.317005,0.657098,0.427160,0.425160,0.399407,0.636641
8,DiasABP,2.660856,2.679686,2.591548,2.938979,2.603837,2.461246,3.028120,2.090026,3.031109,2.554422,2.928648,3.498162,2.361409,2.522613,2.592823,3.451068,2.134358
9,FiO2,0.064674,0.064362,0.073091,0.000000,0.066699,0.065896,0.070376,0.071671,0.070834,0.057199,0.065378,0.078168,0.064161,0.060026,0.065281,0.064115,0.087396


<h5>Minimum MAE value in each subgroup</h5>

In [97]:
toolkits.min_value_in_subgroup(df_brits_mae_standard_ori, subgroups, variables)

General
MechVent
0.007398843992015586
--------------------
Female
MechVent
0.006747702430468682
--------------------
Male
MechVent
0.007949417122953923
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.006393708959893056
--------------------
-65
MechVent
0.006916099086335169
--------------------
ICUType 1
MechVent
0.008637717301896946
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.007601120200842905
--------------------
ICUType 4
MechVent
0.0071316264994522715
--------------------
Undefined classification
MechVent
0.007929448947190589
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.006744161156788904
--------------------
Overweight
MechVent
0.005137603359612789
--------------------
Obesity 1
MechVent
0.0050724968725804846
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [98]:
toolkits.max_value_in_subgroup(df_brits_mae_standard_ori, subgroups, variables) 

General
AST
487.83971088346965
--------------------
Female
AST
530.7552405407484
--------------------
Male
AST
213.84651056483742
--------------------
Undefined Gender
Urine
68.5178756713182
--------------------
+65
AST
197.0986525919505
--------------------
-65
ALT
320.9938058715234
--------------------
ICUType 1
AST
582.2399769973522
--------------------
ICUType 2
ALT
396.22568752889055
--------------------
ICUType 3
AST
417.15465731148737
--------------------
ICUType 4
AST
495.52234619313117
--------------------
Undefined classification
AST
230.8361425650723
--------------------
Low Weight
AST
760.9081611631399
--------------------
Normal Weight
AST
178.00203657149711
--------------------
Overweight
ALT
507.83618397181476
--------------------
Obesity 1
AST
281.5323677062848
--------------------
Obesity 2
ALT
618.4489571706706
--------------------
Obesity 3
ALT
1125.5628045396231
--------------------


<h4> MinMax Scaler (C/Normalização)</h4>

In [99]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_minmax, subgroups, variables)

BRITS - MAE
************
General
-------------
ALP : 0.040219907134693526
ALT : 0.017229766448887466
AST : 0.015468583460460567
Albumin : 0.45373933730150834
BUN : 0.11917518228086253
Bilirubin : 0.04461143951106635
Cholesterol : 0.38712575286624884
Creatinine : 0.06330636563215755
DiasABP : 0.2222883897032941
FiO2 : 0.4212544503171473
GCS : 0.7013794357043793
Glucose : 0.08704999971751116
HCO3 : 0.38295172317766624
HCT : 0.45079366319242514
HR : 0.43347807748668743
K : 0.12318451615964963
Lactate : 0.08926922807197073
MAP : 0.2694956399200512
MechVent : 0.0
Mg : 0.06937552593284982
NIDiasABP : 0.34968573475266584
NIMAP : 0.34092245932783305
NISysABP : 0.45429145035202567
Na : 0.4986517947522833
PaCO2 : 0.4095278180110242
PaO2 : 0.29658351907452635
Platelets : 0.08119226622028226
RespRate : 0.19828358249685923
SaO2 : 0.9685085946654024
SysABP : 0.42148474257749563
Temp : 0.9335184893617962
TroponinI : 0.16517311730421255
TroponinT : 0.02879763797807984
Urine : 0.010554451556121645
WBC 

In [100]:
df_brits_mae_minmax = toolkits.create_table(testing_mae_brits_variables_minmax, subgroups, variables)

In [101]:
df_brits_mae_minmax 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.040220,0.041253,0.042669,0.000000,0.045370,0.058368,0.029260,0.018007,0.063298,0.040524,0.036990,0.128495,0.039369,0.032560,0.033538,0.019912,0.031346
1,ALT,0.017230,0.006941,0.016567,0.000000,0.012877,0.023744,0.018795,0.029335,0.023352,0.016529,0.017182,0.012010,0.009226,0.037875,0.014711,0.045587,0.099730
2,AST,0.015469,0.017081,0.006773,0.000000,0.006469,0.011712,0.018681,0.008078,0.013382,0.017210,0.007568,0.024316,0.004764,0.018236,0.010571,0.002253,0.001133
3,Albumin,0.453739,0.423997,0.450085,0.000000,0.428799,0.450342,0.544186,0.392027,0.419730,0.411677,0.404592,0.488372,0.380661,0.470012,0.461240,0.406977,0.390698
4,BUN,0.119175,0.115023,0.134813,0.000000,0.147150,0.102920,0.136926,0.092971,0.158645,0.091490,0.125755,0.143510,0.128078,0.122560,0.140897,0.149504,0.141484
5,Bilirubin,0.044611,0.034648,0.036118,0.000000,0.026320,0.053072,0.018808,0.037440,0.039132,0.045804,0.044419,0.133333,0.055639,0.025541,0.041761,0.026481,0.035695
6,Cholesterol,0.387126,0.473054,0.389912,0.000000,0.363024,0.238024,0.430447,0.000000,0.300150,0.356287,0.427146,0.377246,0.258982,0.533790,0.311377,0.407186,0.000000
7,Creatinine,0.063306,0.048783,0.072079,0.000000,0.057820,0.060337,0.079545,0.051091,0.086067,0.046976,0.065030,0.041558,0.068246,0.056569,0.057444,0.059394,0.076540
8,DiasABP,0.222288,0.221258,0.223552,0.229701,0.212173,0.235335,0.217357,0.208378,0.226277,0.231468,0.230916,0.218497,0.215240,0.216017,0.220501,0.222197,0.221234
9,FiO2,0.421254,0.407955,0.441474,0.000000,0.431625,0.422815,0.415464,0.454358,0.438086,0.399443,0.399171,0.374807,0.413956,0.412962,0.471615,0.444020,0.479529


<h5>Minimum MAE value in each subgroup</h5>

In [102]:
toolkits.min_value_in_subgroup(df_brits_mae_minmax, subgroups, variables)

General
MechVent
0.0
--------------------
Female
MechVent
0.0
--------------------
Male
MechVent
0.0
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.0
--------------------
-65
MechVent
0.0
--------------------
ICUType 1
MechVent
0.0
--------------------
ICUType 2
MechVent
0.0
--------------------
ICUType 3
MechVent
0.0
--------------------
ICUType 4
MechVent
0.0
--------------------
Undefined classification
MechVent
0.0
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.0
--------------------
Overweight
MechVent
0.0
--------------------
Obesity 1
MechVent
0.0
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
MechVent
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [103]:
toolkits.max_value_in_subgroup(df_brits_mae_minmax, subgroups, variables) 

General
SaO2
0.9685085946654024
--------------------
Female
SaO2
0.9617415842380362
--------------------
Male
SaO2
0.9675699026353866
--------------------
Undefined Gender
GCS
0.9375000149009268
--------------------
+65
SaO2
0.9675195857723378
--------------------
-65
SaO2
0.9603608350471073
--------------------
ICUType 1
SaO2
0.964683557990219
--------------------
ICUType 2
SaO2
0.9694730503594138
--------------------
ICUType 3
Temp
0.9274873339532732
--------------------
ICUType 4
SaO2
0.9657142966512496
--------------------
Undefined classification
SaO2
0.9598484974015699
--------------------
Low Weight
SaO2
0.966071435383319
--------------------
Normal Weight
SaO2
0.9706250113936478
--------------------
Overweight
SaO2
0.9674803281393501
--------------------
Obesity 1
SaO2
0.9677868954470741
--------------------
Obesity 2
SaO2
0.962777776850541
--------------------
Obesity 3
SaO2
0.9603703794655621
--------------------


<h4> MinMax Scaler (S/Normalização)</h4>

In [104]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_minmax_ori, subgroups, variables)

BRITS - MAE
************
General
-------------
ALP : 120.21354016661581
ALT : 292.51042278607537
AST : 566.9945693689813
Albumin : 2.9510791413217787
BUN : 25.788438117812007
Bilirubin : 2.7703703989979305
Cholesterol : 157.29999999999214
Creatinine : 1.4927400290198274
DiasABP : 59.684729736472974
FiO2 : 0.5427910102481897
GCS : 11.416553277655188
Glucose : 139.70664855678496
HCO3 : 22.998730468265872
HCT : 30.609207015520653
HR : 86.69561557043218
K : 4.13614858243916
Lactate : 2.767346140115878
MAP : 80.30970073528962
MechVent : 0.9999999999999996
Mg : 2.011890264982128
NIDiasABP : 58.44698860910438
NIMAP : 77.73032076927599
NISysABP : 119.4786514127118
Na : 138.88944723618073
PaCO2 : 40.95278192858044
PaO2 : 148.29175956176658
Platelets : 190.6867094356714
RespRate : 19.431791012219442
SaO2 : 96.8508583690985
SysABP : 120.12315150566705
Temp : 37.09088744652194
TroponinI : 8.210000050067492
TroponinT : 0.8710493570492538
Urine : 116.09896736077816
WBC : 12.26488773373189
Weight : 8

In [105]:
df_brits_mae_minmax_ori = toolkits.create_table(testing_mae_brits_variables_minmax_ori, subgroups, variables)

In [106]:
df_brits_mae_minmax_ori 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,120.213540,123.095891,127.046296,0.000000,134.583333,170.847619,89.636364,58.240000,184.601941,121.060607,111.202248,366.500000,117.838710,98.843750,101.571429,63.555556,95.454543
1,ALT,292.510423,118.436620,281.300001,0.000000,218.868130,402.716417,319.000002,497.315789,396.091955,280.655731,291.708863,204.200000,157.095240,641.805538,249.900001,772.285714,1688.333333
2,AST,566.994569,625.684209,250.518520,0.000000,239.448055,430.263396,683.920050,298.000001,491.054948,630.363636,279.442109,889.000068,177.375000,667.714272,388.750004,86.000001,45.250000
3,Albumin,2.951079,2.823188,2.935366,0.000000,2.843836,2.936471,3.340000,2.685714,2.804839,2.770213,2.739744,3.100000,2.636842,3.021053,2.983333,2.750000,2.680000
4,BUN,25.788438,24.924859,29.041096,0.000000,31.607143,22.407408,29.480620,20.337931,33.998246,20.029915,27.157143,30.850000,27.640288,26.492424,30.306667,32.096774,30.428572
5,Bilirubin,2.770370,2.151613,2.242935,0.000000,1.634444,3.295798,1.168000,2.325000,2.430097,2.844444,2.758427,8.280000,3.455172,1.586111,2.593333,1.644445,2.216667
6,Cholesterol,157.300000,185.999994,158.230769,0.000000,149.250000,107.500000,171.769229,0.000000,128.250000,147.000000,170.666665,154.000000,114.499992,206.285712,132.000000,163.999985,0.000000
7,Creatinine,1.492740,1.173219,1.685744,0.000000,1.372038,1.427404,1.850000,1.224000,1.993464,1.133466,1.530662,1.014286,1.601408,1.344526,1.363768,1.406667,1.783871
8,DiasABP,59.684730,59.403303,60.029592,61.708333,56.923331,63.246397,58.338428,55.887238,60.773501,62.190792,62.040018,58.649750,57.760640,57.972569,59.196723,59.659678,59.396970
9,FiO2,0.542791,0.532284,0.558764,0.000000,0.550984,0.544024,0.538217,0.568943,0.556088,0.525560,0.525345,0.506098,0.537025,0.536240,0.582576,0.560776,0.588828


<h5>Minimum MAE value in each subgroup</h5>

In [107]:
toolkits.min_value_in_subgroup(df_brits_mae_minmax_ori, subgroups, variables)

General
FiO2
0.5427910102481897
--------------------
Female
FiO2
0.5322841955806464
--------------------
Male
FiO2
0.558764148292676
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
FiO2
0.5509840135837525
--------------------
-65
FiO2
0.5440240574521423
--------------------
ICUType 1
FiO2
0.5382165582316663
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
TroponinT
0.4359321938985411
--------------------
ICUType 4
TroponinT
0.4575000104183762
--------------------
Undefined classification
FiO2
0.5253448272903851
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
FiO2
0.5370249722019451
--------------------
Overweight
FiO2
0.5362397804247246
--------------------
Obesity 1
FiO2
0.5825757534816982
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [108]:
toolkits.max_value_in_subgroup(df_brits_mae_minmax_ori, subgroups, variables) 

General
AST
566.9945693689813
--------------------
Female
AST
625.6842090706995
--------------------
Male
ALT
281.3000011026836
--------------------
Undefined Gender
Urine
139.99999999986
--------------------
+65
AST
239.44805549026773
--------------------
-65
AST
430.2633963482682
--------------------
ICUType 1
AST
683.9200496291841
--------------------
ICUType 2
ALT
497.31578862037054
--------------------
ICUType 3
AST
491.05494833516013
--------------------
ICUType 4
AST
630.3636355139878
--------------------
Undefined classification
ALT
291.7088629807061
--------------------
Low Weight
AST
889.0000677106542
--------------------
Normal Weight
AST
177.37499982118052
--------------------
Overweight
AST
667.7142724082266
--------------------
Obesity 1
AST
388.75000405309646
--------------------
Obesity 2
ALT
772.2857140131256
--------------------
Obesity 3
ALT
1688.3333333327705
--------------------


#### USGAN 

<h4> Standard Scaler (C/Normalização)</h4>

In [109]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_standard, subgroups, variables)

USGAN - MAE
************
General
-------------
ALP : 0.4830952143501289
ALT : 0.23563931085337708
AST : 0.2858888140250534
Albumin : 0.6459737302091527
BUN : 0.2952975500709421
Bilirubin : 0.4779943941181737
Cholesterol : 1.7866300252159824
Creatinine : 0.3653507985392498
DiasABP : 0.2539470160847463
FiO2 : 0.3937069756002726
GCS : 0.23771497167583566
Glucose : 0.5876730509070685
HCO3 : 0.43356488242200864
HCT : 0.48827926561613155
HR : 0.24464453507626363
K : 0.5923851052202248
Lactate : 0.34453583359235085
MAP : 0.20323457182695304
MechVent : 0.04214297756008579
Mg : 0.5543555867984249
NIDiasABP : 0.21009112576624886
NIMAP : 0.13953849345883035
NISysABP : 0.28515123738917225
Na : 0.4189623117918906
PaCO2 : 0.5265201095645239
PaO2 : 0.4839426425570712
Platelets : 0.4075142522848474
RespRate : 0.4788782491025623
SaO2 : 0.38531693193499345
SysABP : 0.2859506529445208
Temp : 0.24285885388206577
TroponinI : 0.8455114102832897
TroponinT : 0.4879422214949129
Urine : 0.3522886338174635
WBC :

In [110]:
df_usgan_mae_standard = toolkits.create_table(testing_mae_saits_variables_standard, subgroups, variables)

In [111]:
df_usgan_mae_standard 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.355492,0.398791,0.388989,0.000000,0.331751,0.740879,0.222358,0.195952,0.723086,0.505640,0.296971,0.216395,0.284060,0.236059,0.184009,0.347203,0.266653
1,ALT,0.346164,0.267721,0.253539,0.000000,0.280160,0.329866,0.296968,0.351267,0.311489,0.272558,0.325780,0.238700,0.251692,0.305938,0.274798,0.318639,0.823624
2,AST,0.283204,0.257204,0.204749,0.000000,0.206225,0.243144,0.340113,0.273045,0.305834,0.306141,0.244520,0.699315,0.218977,0.318423,0.207619,0.237748,0.209382
3,Albumin,0.586538,0.609286,0.621456,0.000000,0.536556,0.658661,0.490084,0.717311,0.708582,0.754085,0.776171,1.172093,0.840292,0.814925,0.467807,0.670209,0.729714
4,BUN,0.235495,0.253363,0.228878,0.000000,0.242391,0.244288,0.213308,0.222898,0.294829,0.209661,0.257893,0.200723,0.219590,0.235670,0.210442,0.309354,0.251667
5,Bilirubin,0.292134,0.301597,0.249420,0.000000,0.231827,0.336409,0.200403,0.318250,0.280385,0.267887,0.312005,0.954099,0.392959,0.242916,0.335206,0.310083,0.216768
6,Cholesterol,0.861914,1.451439,0.918617,0.000000,0.736495,1.183795,1.002457,0.000000,0.906296,0.725019,0.618523,0.085870,0.704775,1.232571,0.533449,0.347422,0.000000
7,Creatinine,0.227236,0.218419,0.210438,0.000000,0.200026,0.211177,0.193449,0.183120,0.265242,0.217446,0.233475,0.177189,0.227362,0.206401,0.179911,0.192612,0.192658
8,DiasABP,0.256964,0.260941,0.240924,0.182851,0.271154,0.231552,0.299906,0.221175,0.250333,0.241564,0.250752,0.246645,0.246105,0.255920,0.251888,0.235695,0.250253
9,FiO2,0.484346,0.452276,0.517671,0.000000,0.491061,0.501327,0.482516,0.467761,0.507122,0.438264,0.460432,0.476691,0.439314,0.414027,0.507958,0.501128,0.573521


<h5>Minimum MAE value in each subgroup</h5>

In [112]:
toolkits.min_value_in_subgroup(df_usgan_mae_standard, subgroups, variables)

General
MechVent
0.016131509283529688
--------------------
Female
MechVent
0.014552340041130598
--------------------
Male
MechVent
0.014512000309648741
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.012942020909290175
--------------------
-65
MechVent
0.01827778097688463
--------------------
ICUType 1
MechVent
0.012979781499681835
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.015398824789147038
--------------------
ICUType 4
MechVent
0.01676655673093299
--------------------
Undefined classification
MechVent
0.015602762801210616
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.013784697335756175
--------------------
Overweight
MechVent
0.014702130836156856
--------------------
Obesity 1
MechVent
0.014115385650734221
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [113]:
toolkits.max_value_in_subgroup(df_usgan_mae_standard, subgroups, variables)

General
Cholesterol
0.8619142944054452
--------------------
Female
Cholesterol
1.451438736858023
--------------------
Male
Cholesterol
0.9186172160150028
--------------------
Undefined Gender
K
0.6481980955579346
--------------------
+65
TroponinI
0.912479961059112
--------------------
-65
Cholesterol
1.1837952493687076
--------------------
ICUType 1
TroponinI
1.4147481358654532
--------------------
ICUType 2
Albumin
0.7173112394561357
--------------------
ICUType 3
SaO2
1.5843908865861114
--------------------
ICUType 4
TroponinI
0.8606517177368178
--------------------
Undefined classification
Albumin
0.7761709655447225
--------------------
Low Weight
Albumin
1.1720931746200332
--------------------
Normal Weight
Albumin
0.8402915751610249
--------------------
Overweight
Cholesterol
1.2325708781179912
--------------------
Obesity 1
Ph
1.006255796754588
--------------------
Obesity 2
TroponinT
1.4516069413777122
--------------------
Obesity 3
TroponinT
1.03692539683769
------------------

<h4> Standard Scaler (S/Normalização)</h4>

In [114]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_standard_ori, subgroups, variables)

USGAN - MAE
************
General
-------------
ALP : 62.582011178135545
ALT : 276.51446573948346
AST : 501.7037326205171
Albumin : 0.42421031444192486
BUN : 6.74664816220014
Bilirubin : 2.7384976725019965
Cholesterol : 79.59944438933928
Creatinine : 0.5633740507441389
DiasABP : 3.2476638301528746
FiO2 : 0.07400986563016994
GCS : 0.9510791854121006
Glucose : 37.60932279783241
HCO3 : 2.050341823984521
HCT : 2.450253662579908
HR : 4.3501495309163145
K : 0.40185893199465256
Lactate : 0.9161802539450833
MAP : 3.2227640694362245
MechVent : 0.042142977403557796
Mg : 0.26909216334180097
NIDiasABP : 3.1508690714875307
NIMAP : 2.0786575146979573
NISysABP : 6.323983373312606
Na : 2.197734353530344
PaCO2 : 4.8006159194566465
PaO2 : 41.302955348489874
Platelets : 43.84992549236419
RespRate : 2.5706728400285925
SaO2 : 1.3253946427111944
SysABP : 6.7008276819947215
Temp : 0.30990340187101834
TroponinI : 9.366745508312713
TroponinT : 1.4692467343602469
Urine : 55.10455513474912
WBC : 5.599973036490147

In [115]:
df_usgan_mae_standard_ori = toolkits.create_table(testing_mae_usgan_variables_standard_ori, subgroups, variables)

In [116]:
df_usgan_mae_standard_ori

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,62.582011,62.976859,72.131072,0.000000,78.546521,140.328056,34.280117,26.792334,148.879078,66.596595,49.086859,321.526497,70.545888,41.653822,43.584446,33.468035,40.330052
1,ALT,276.514466,203.817833,240.834035,0.000000,236.705691,323.471745,320.351934,218.774189,430.367580,245.551464,224.667079,199.851144,246.953181,381.463462,232.783405,250.800637,191.477491
2,AST,501.703733,421.049596,313.640503,0.000000,236.050086,387.333161,638.485120,284.152509,464.078604,491.976613,330.782198,598.578918,322.291564,377.444810,266.258398,279.410772,251.673035
3,Albumin,0.424210,0.418162,0.410935,0.000000,0.331869,0.477397,0.358058,0.567833,0.506614,0.512721,0.543184,0.461499,0.541082,0.457659,0.471659,0.352888,0.331039
4,BUN,6.746648,7.598816,7.390895,0.000000,7.335730,6.901404,7.436832,5.506618,9.420850,6.016153,7.565286,6.057080,7.530353,6.739874,8.307727,7.746777,7.768571
5,Bilirubin,2.738498,2.113708,2.101996,0.000000,1.846629,3.082248,1.865937,2.574042,2.824355,2.330955,2.979575,9.169635,3.424610,1.998443,2.577595,1.890248,2.011379
6,Cholesterol,79.599444,117.885786,90.495092,0.000000,81.115471,58.796846,92.764605,0.000000,81.540899,98.721514,93.746525,86.495720,73.802685,107.636538,58.767792,96.887329,0.000000
7,Creatinine,0.563374,0.432088,0.627338,0.000000,0.474322,0.607357,0.683837,0.443779,0.857093,0.436033,0.691229,0.382321,0.679678,0.443356,0.591183,0.438260,0.645646
8,DiasABP,3.247664,3.275051,3.198573,2.980937,3.314258,3.081312,3.508510,2.738938,3.325169,3.258443,3.453536,3.441222,2.993881,3.152988,3.106209,3.446952,2.957783
9,FiO2,0.074010,0.072352,0.079735,0.000000,0.071523,0.076751,0.070184,0.073121,0.079996,0.069943,0.073352,0.075876,0.068607,0.066190,0.071889,0.076006,0.093829


<h5>Minimum MAE value in each subgroup</h5>

In [117]:
toolkits.min_value_in_subgroup(df_usgan_mae_standard_ori, subgroups, variables)

General
MechVent
0.042142977403557796
--------------------
Female
MechVent
0.044038528849885565
--------------------
Male
MechVent
0.0423781132625591
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.039896660022907385
--------------------
-65
MechVent
0.041597186969721874
--------------------
ICUType 1
MechVent
0.037152298062825106
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.045918399773313766
--------------------
ICUType 4
MechVent
0.04235743006866401
--------------------
Undefined classification
MechVent
0.04091206184980371
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.043431381872095075
--------------------
Overweight
MechVent
0.040878757050162744
--------------------
Obesity 1
MechVent
0.03858277514494564
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [118]:
toolkits.max_value_in_subgroup(df_usgan_mae_standard_ori, subgroups, variables)

General
AST
501.7037326205171
--------------------
Female
AST
421.04959572615826
--------------------
Male
AST
313.6405033774093
--------------------
Undefined Gender
Urine
25.44162750241596
--------------------
+65
ALT
236.7056905096678
--------------------
-65
AST
387.33316143069607
--------------------
ICUType 1
AST
638.4851197814686
--------------------
ICUType 2
AST
284.15250894296497
--------------------
ICUType 3
AST
464.078604373298
--------------------
ICUType 4
AST
491.9766127152787
--------------------
Undefined classification
AST
330.7821978877313
--------------------
Low Weight
AST
598.5789184568816
--------------------
Normal Weight
AST
322.2915635704893
--------------------
Overweight
ALT
381.4634622997602
--------------------
Obesity 1
AST
266.25839843748673
--------------------
Obesity 2
AST
279.4107716877324
--------------------
Obesity 3
AST
251.67303466790582
--------------------


<h4>MinMax Scaler (C/Normalização)</h4>

In [119]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_minmax, subgroups, variables)

USGAN - MAE
************
General
-------------
ALP : 0.040219907134693526
ALT : 0.017229766448887466
AST : 0.015468583460460567
Albumin : 0.45373933730150834
BUN : 0.11917518228086253
Bilirubin : 0.04461143951106635
Cholesterol : 0.38712575286624884
Creatinine : 0.06330636563215755
DiasABP : 0.2222883897032941
FiO2 : 0.4212544503171473
GCS : 0.7013794357043793
Glucose : 0.08704999971751116
HCO3 : 0.38295172317766624
HCT : 0.45079366319242514
HR : 0.43347807748668743
K : 0.12318451615964963
Lactate : 0.08926922807197073
MAP : 0.2694956399200512
MechVent : 0.0
Mg : 0.06937552593284982
NIDiasABP : 0.34968573475266584
NIMAP : 0.34092245932783305
NISysABP : 0.45429145035202567
Na : 0.4986517947522833
PaCO2 : 0.4095278180110242
PaO2 : 0.29658351907452635
Platelets : 0.08119226622028226
RespRate : 0.19828358249685923
SaO2 : 0.9685085946654024
SysABP : 0.42148474257749563
Temp : 0.9335184893617962
TroponinI : 0.16517311730421255
TroponinT : 0.02879763797807984
Urine : 0.010554451556121645
WBC 

In [120]:
df_usgan_mae_minmax = toolkits.create_table(testing_mae_usgan_variables_minmax, subgroups, variables)

In [121]:
df_usgan_mae_minmax

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.040220,0.041253,0.042669,0.000000,0.045370,0.058368,0.029260,0.018007,0.063298,0.040524,0.036990,0.128495,0.039369,0.032560,0.033538,0.019912,0.031346
1,ALT,0.017230,0.006941,0.016567,0.000000,0.012877,0.023744,0.018795,0.029335,0.023352,0.016529,0.017182,0.012010,0.009226,0.037875,0.014711,0.045587,0.099730
2,AST,0.015469,0.017081,0.006773,0.000000,0.006469,0.011712,0.018681,0.008078,0.013382,0.017210,0.007568,0.024316,0.004764,0.018236,0.010571,0.002253,0.001133
3,Albumin,0.453739,0.423997,0.450085,0.000000,0.428799,0.450342,0.544186,0.392027,0.419730,0.411677,0.404592,0.488372,0.380661,0.470012,0.461240,0.406977,0.390698
4,BUN,0.119175,0.115023,0.134813,0.000000,0.147150,0.102920,0.136926,0.092971,0.158645,0.091490,0.125755,0.143510,0.128078,0.122560,0.140897,0.149504,0.141484
5,Bilirubin,0.044611,0.034648,0.036118,0.000000,0.026320,0.053072,0.018808,0.037440,0.039132,0.045804,0.044419,0.133333,0.055639,0.025541,0.041761,0.026481,0.035695
6,Cholesterol,0.387126,0.473054,0.389912,0.000000,0.363024,0.238024,0.430447,0.000000,0.300150,0.356287,0.427146,0.377246,0.258982,0.533790,0.311377,0.407186,0.000000
7,Creatinine,0.063306,0.048783,0.072079,0.000000,0.057820,0.060337,0.079545,0.051091,0.086067,0.046976,0.065030,0.041558,0.068246,0.056569,0.057444,0.059394,0.076540
8,DiasABP,0.222288,0.221258,0.223552,0.229701,0.212173,0.235335,0.217357,0.208378,0.226277,0.231468,0.230916,0.218497,0.215240,0.216017,0.220501,0.222197,0.221234
9,FiO2,0.421254,0.407955,0.441474,0.000000,0.431625,0.422815,0.415464,0.454358,0.438086,0.399443,0.399171,0.374807,0.413956,0.412962,0.471615,0.444020,0.479529


<h5>Minimum MAE value in each subgroup</h5>

In [122]:
toolkits.min_value_in_subgroup(df_usgan_mae_minmax, subgroups, variables)

General
MechVent
0.0
--------------------
Female
MechVent
0.0
--------------------
Male
MechVent
0.0
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.0
--------------------
-65
MechVent
0.0
--------------------
ICUType 1
MechVent
0.0
--------------------
ICUType 2
MechVent
0.0
--------------------
ICUType 3
MechVent
0.0
--------------------
ICUType 4
MechVent
0.0
--------------------
Undefined classification
MechVent
0.0
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.0
--------------------
Overweight
MechVent
0.0
--------------------
Obesity 1
MechVent
0.0
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
MechVent
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [123]:
toolkits.max_value_in_subgroup(df_usgan_mae_minmax, subgroups, variables)

General
SaO2
0.9685085946654024
--------------------
Female
SaO2
0.9617415842380362
--------------------
Male
SaO2
0.9675699026353866
--------------------
Undefined Gender
GCS
0.9375000149009268
--------------------
+65
SaO2
0.9675195857723378
--------------------
-65
SaO2
0.9603608350471073
--------------------
ICUType 1
SaO2
0.964683557990219
--------------------
ICUType 2
SaO2
0.9694730503594138
--------------------
ICUType 3
Temp
0.9274873339532732
--------------------
ICUType 4
SaO2
0.9657142966512496
--------------------
Undefined classification
SaO2
0.9598484974015699
--------------------
Low Weight
SaO2
0.966071435383319
--------------------
Normal Weight
SaO2
0.9706250113936478
--------------------
Overweight
SaO2
0.9674803281393501
--------------------
Obesity 1
SaO2
0.9677868954470741
--------------------
Obesity 2
SaO2
0.962777776850541
--------------------
Obesity 3
SaO2
0.9603703794655621
--------------------


<h4>MinMax Scaler (S/Normalização)</h4>

In [124]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_minmax_ori, subgroups, variables)

USGAN - MAE
************
General
-------------
ALP : 120.21354016661581
ALT : 292.51042278607537
AST : 566.9945693689813
Albumin : 2.9510791413217787
BUN : 25.788438117812007
Bilirubin : 2.7703703989979305
Cholesterol : 157.29999999999214
Creatinine : 1.4927400290198274
DiasABP : 59.684729736472974
FiO2 : 0.5427910102481897
GCS : 11.416553277655188
Glucose : 139.70664855678496
HCO3 : 22.998730468265872
HCT : 30.609207015520653
HR : 86.69561557043218
K : 4.13614858243916
Lactate : 2.767346140115878
MAP : 80.30970073528962
MechVent : 0.9999999999999996
Mg : 2.011890264982128
NIDiasABP : 58.44698860910438
NIMAP : 77.73032076927599
NISysABP : 119.4786514127118
Na : 138.88944723618073
PaCO2 : 40.95278192858044
PaO2 : 148.29175956176658
Platelets : 190.6867094356714
RespRate : 19.431791012219442
SaO2 : 96.8508583690985
SysABP : 120.12315150566705
Temp : 37.09088744652194
TroponinI : 8.210000050067492
TroponinT : 0.8710493570492538
Urine : 116.09896736077816
WBC : 12.26488773373189
Weight : 8

In [125]:
df_usgan_mae_minmax_ori = toolkits.create_table(testing_mae_usgan_variables_minmax_ori, subgroups, variables)

In [126]:
df_usgan_mae_minmax_ori

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,120.213540,123.095891,127.046296,0.000000,134.583333,170.847619,89.636364,58.240000,184.601941,121.060607,111.202248,366.500000,117.838710,98.843750,101.571429,63.555556,95.454543
1,ALT,292.510423,118.436620,281.300001,0.000000,218.868130,402.716417,319.000002,497.315789,396.091955,280.655731,291.708863,204.200000,157.095240,641.805538,249.900001,772.285714,1688.333333
2,AST,566.994569,625.684209,250.518520,0.000000,239.448055,430.263396,683.920050,298.000001,491.054948,630.363636,279.442109,889.000068,177.375000,667.714272,388.750004,86.000001,45.250000
3,Albumin,2.951079,2.823188,2.935366,0.000000,2.843836,2.936471,3.340000,2.685714,2.804839,2.770213,2.739744,3.100000,2.636842,3.021053,2.983333,2.750000,2.680000
4,BUN,25.788438,24.924859,29.041096,0.000000,31.607143,22.407408,29.480620,20.337931,33.998246,20.029915,27.157143,30.850000,27.640288,26.492424,30.306667,32.096774,30.428572
5,Bilirubin,2.770370,2.151613,2.242935,0.000000,1.634444,3.295798,1.168000,2.325000,2.430097,2.844444,2.758427,8.280000,3.455172,1.586111,2.593333,1.644445,2.216667
6,Cholesterol,157.300000,185.999994,158.230769,0.000000,149.250000,107.500000,171.769229,0.000000,128.250000,147.000000,170.666665,154.000000,114.499992,206.285712,132.000000,163.999985,0.000000
7,Creatinine,1.492740,1.173219,1.685744,0.000000,1.372038,1.427404,1.850000,1.224000,1.993464,1.133466,1.530662,1.014286,1.601408,1.344526,1.363768,1.406667,1.783871
8,DiasABP,59.684730,59.403303,60.029592,61.708333,56.923331,63.246397,58.338428,55.887238,60.773501,62.190792,62.040018,58.649750,57.760640,57.972569,59.196723,59.659678,59.396970
9,FiO2,0.542791,0.532284,0.558764,0.000000,0.550984,0.544024,0.538217,0.568943,0.556088,0.525560,0.525345,0.506098,0.537025,0.536240,0.582576,0.560776,0.588828


<h5>Minimum MAE value in each subgroup</h5>

In [127]:
toolkits.min_value_in_subgroup(df_usgan_mae_minmax_ori, subgroups, variables)

General
FiO2
0.5427910102481897
--------------------
Female
FiO2
0.5322841955806464
--------------------
Male
FiO2
0.558764148292676
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
FiO2
0.5509840135837525
--------------------
-65
FiO2
0.5440240574521423
--------------------
ICUType 1
FiO2
0.5382165582316663
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
TroponinT
0.4359321938985411
--------------------
ICUType 4
TroponinT
0.4575000104183762
--------------------
Undefined classification
FiO2
0.5253448272903851
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
FiO2
0.5370249722019451
--------------------
Overweight
FiO2
0.5362397804247246
--------------------
Obesity 1
FiO2
0.5825757534816982
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [128]:
toolkits.max_value_in_subgroup(df_usgan_mae_minmax_ori, subgroups, variables)

General
AST
566.9945693689813
--------------------
Female
AST
625.6842090706995
--------------------
Male
ALT
281.3000011026836
--------------------
Undefined Gender
Urine
139.99999999986
--------------------
+65
AST
239.44805549026773
--------------------
-65
AST
430.2633963482682
--------------------
ICUType 1
AST
683.9200496291841
--------------------
ICUType 2
ALT
497.31578862037054
--------------------
ICUType 3
AST
491.05494833516013
--------------------
ICUType 4
AST
630.3636355139878
--------------------
Undefined classification
ALT
291.7088629807061
--------------------
Low Weight
AST
889.0000677106542
--------------------
Normal Weight
AST
177.37499982118052
--------------------
Overweight
AST
667.7142724082266
--------------------
Obesity 1
AST
388.75000405309646
--------------------
Obesity 2
ALT
772.2857140131256
--------------------
Obesity 3
ALT
1688.3333333327705
--------------------


#### GPVAE

<h4> Standard Scaler (C/Normalização)</h4>

In [129]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_standard, subgroups, variables)

GPVAE - MAE
************
General
-------------
ALP : 0.5091427052481455
ALT : 0.3990662880330406
AST : 0.4462108080752909
Albumin : 0.7059858721445161
BUN : 0.608651836670855
Bilirubin : 0.4721013517081156
Cholesterol : 0.8939011898098311
Creatinine : 0.5800814022156039
DiasABP : 0.46682311076058514
FiO2 : 0.6648206271366965
GCS : 0.4951801829177615
Glucose : 0.5920130879814554
HCO3 : 0.6884326100719589
HCT : 0.7431230082588463
HR : 0.3569348183100172
K : 0.6921673643592008
Lactate : 0.47356833825548744
MAP : 0.4212884518757922
MechVent : 0.0041282574803201124
Mg : 0.605049392106839
NIDiasABP : 0.527148528007825
NIMAP : 0.45653561724442504
NISysABP : 0.4920234309751518
Na : 0.6867624517981127
PaCO2 : 0.6732264320230728
PaO2 : 0.5817960074877802
Platelets : 0.7310812978366584
RespRate : 0.5922825256486578
SaO2 : 0.41121382956948066
SysABP : 0.5061041632267137
Temp : 0.4531240937684892
TroponinI : 0.713798682464535
TroponinT : 0.3826653059345242
Urine : 0.4349922716549518
WBC : 0.1282555

In [130]:
df_gpvae_mae_standard = toolkits.create_table(testing_mae_gpvae_variables_standard, subgroups, variables)

In [131]:
df_gpvae_mae_standard 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.509143,0.497518,0.577839,0.000000,0.689788,1.119264,0.293807,0.408193,1.152444,0.528068,0.462810,2.332871,0.563558,0.342092,0.319656,0.318552,0.211467
1,ALT,0.399066,0.291641,0.329594,0.000000,0.297686,0.506219,0.391648,0.482915,0.537038,0.380928,0.410370,0.235083,0.323138,0.629439,0.238210,0.727262,1.321828
2,AST,0.446211,0.471410,0.307604,0.000000,0.275551,0.400831,0.477714,0.309292,0.420884,0.451580,0.254051,0.648292,0.304592,0.473809,0.304970,0.183026,0.198119
3,Albumin,0.705986,0.673233,0.652846,0.000000,0.561812,0.776023,0.645771,0.875697,0.751691,0.802882,0.839060,0.683913,0.759627,0.737423,0.607951,0.710335,0.582020
4,BUN,0.608652,0.611320,0.653697,0.000000,0.674840,0.579672,0.582594,0.667353,0.680541,0.554125,0.636820,0.567308,0.640443,0.661506,0.678634,0.924292,0.677526
5,Bilirubin,0.472101,0.450318,0.405089,0.000000,0.403495,0.520059,0.279062,0.519343,0.435798,0.467214,0.505855,1.443795,0.567849,0.379859,0.463863,0.427504,0.457885
6,Cholesterol,0.893901,1.452625,0.723576,0.000000,0.580342,0.898329,0.959251,0.000000,0.597481,0.586300,0.587158,0.031211,0.512781,1.198163,0.359504,0.167166,0.000000
7,Creatinine,0.580081,0.517428,0.589659,0.000000,0.543965,0.517365,0.594157,0.548637,0.692436,0.492750,0.612070,0.600689,0.611044,0.634833,0.479905,0.476310,0.710344
8,DiasABP,0.466823,0.485747,0.448321,0.321478,0.475140,0.446476,0.468639,0.393620,0.520822,0.479730,0.505715,0.542414,0.449825,0.425068,0.423624,0.488934,0.428724
9,FiO2,0.664821,0.641951,0.711987,0.000000,0.665195,0.721725,0.671221,0.661313,0.730535,0.635573,0.636522,0.582704,0.683298,0.627044,0.684677,0.713074,0.662698


<h5>Minimum MAE value in each subgroup</h5>

In [132]:
toolkits.min_value_in_subgroup(df_gpvae_mae_standard, subgroups, variables)

General
MechVent
0.0041282574803201124
--------------------
Female
MechVent
0.004081880589528707
--------------------
Male
MechVent
0.003964355274035993
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.0043638198150457875
--------------------
-65
MechVent
0.003671528340863437
--------------------
ICUType 1
MechVent
0.004535948957644015
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.0038367602691898386
--------------------
ICUType 4
MechVent
0.0037402539388979735
--------------------
Undefined classification
MechVent
0.003936311825521562
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.0047508856876927385
--------------------
Overweight
MechVent
0.00431482755786354
--------------------
Obesity 1
MechVent
0.003922692137141305
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [133]:
toolkits.max_value_in_subgroup(df_gpvae_mae_standard, subgroups, variables)

General
Cholesterol
0.8939011898098311
--------------------
Female
Cholesterol
1.4526252261718635
--------------------
Male
HCT
0.7480109466506408
--------------------
Undefined Gender
K
1.092884754237929
--------------------
+65
TroponinI
1.0471298057925045
--------------------
-65
ALP
1.1192636785858412
--------------------
ICUType 1
TroponinI
1.371157726047459
--------------------
ICUType 2
Albumin
0.8756971129845146
--------------------
ICUType 3
SaO2
1.6505289544534274
--------------------
ICUType 4
TroponinI
0.8368552636648251
--------------------
Undefined classification
PaCO2
0.8414044629276659
--------------------
Low Weight
ALP
2.332871248195961
--------------------
Normal Weight
Na
0.7663294121180635
--------------------
Overweight
Cholesterol
1.1981632166885228
--------------------
Obesity 1
Ph
0.9429491033665625
--------------------
Obesity 2
TroponinT
1.3682938866621464
--------------------
Obesity 3
ALT
1.3218280958769986
--------------------


<h4> Standard Scaler (S/Normalização)</h4>

In [134]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_standard_ori, subgroups, variables)

GPVAE - MAE
************
General
-------------
ALP : 65.95630307992265
ALT : 468.29029258092004
AST : 783.051366422483
Albumin : 0.46362023628014987
BUN : 13.905837754628445
Bilirubin : 2.7047355890273908
Cholesterol : 39.82583923339645
Creatinine : 0.8944904763190462
DiasABP : 5.970082107254754
FiO2 : 0.12497437933360031
GCS : 1.9811775315012357
Glucose : 37.88707169156026
HCO3 : 3.255619276598621
HCT : 3.729095308749158
HR : 6.346840451660973
K : 0.4695486680881391
Lactate : 1.259299968810089
MAP : 6.680523342332905
MechVent : 0.004128255824679912
Mg : 0.2936996673956146
NIDiasABP : 7.905978666268846
NIMAP : 6.800856201709553
NISysABP : 10.911921858420632
Na : 3.602523171122944
PaCO2 : 6.138229939808429
PaO2 : 49.654426886226446
Platelets : 78.66684507040078
RespRate : 3.1794399080010103
SaO2 : 1.414473423118752
SysABP : 11.8597973055738
Temp : 0.5782152376656631
TroponinI : 7.907605323791109
TroponinT : 1.1522465626913752
Urine : 68.04095656948748
WBC : 5.297876364387948
Weight : 5.

In [135]:
df_gpvae_mae_standard_ori = toolkits.create_table(testing_mae_gpvae_variables_standard_ori, subgroups, variables)

In [136]:
df_gpvae_mae_standard_ori 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,65.956303,64.450351,74.855455,0.000000,89.357782,144.993719,38.060928,52.878858,149.291977,68.407936,59.954247,302.209110,73.005477,44.315924,41.409446,41.266463,27.394187
1,ALT,468.290293,342.230013,386.767393,0.000000,349.324061,594.030761,459.584754,566.684399,630.194872,447.006021,481.555024,275.861188,379.191457,738.624324,279.531161,853.416677,1551.118912
2,AST,783.051366,827.273986,539.811579,0.000000,483.561670,703.414177,838.336791,542.772986,738.605910,792.472848,445.831115,1137.682480,534.526317,831.482861,535.188569,321.191401,347.676980
3,Albumin,0.463620,0.442112,0.428724,0.000000,0.368942,0.509613,0.424077,0.575069,0.493635,0.527252,0.551010,0.449125,0.498846,0.484265,0.399241,0.466476,0.382212
4,BUN,13.905838,13.966791,14.934983,0.000000,15.418024,13.243737,13.310493,15.246984,15.548283,12.660070,14.549395,12.961247,14.632174,15.113401,15.504709,21.117253,15.479414
5,Bilirubin,2.704736,2.579933,2.320813,0.000000,2.311681,2.979493,1.598786,2.975390,2.496750,2.676736,2.898117,8.271708,3.253287,2.176264,2.657537,2.449230,2.623289
6,Cholesterol,39.825839,64.718579,32.237375,0.000000,25.855872,40.023094,42.737349,0.000000,26.619488,26.121329,26.159544,1.390549,22.845871,53.381574,16.016902,7.447723,0.000000
7,Creatinine,0.894490,0.797878,0.909258,0.000000,0.838798,0.797781,0.916195,0.846004,1.067742,0.759824,0.943817,0.926268,0.942235,0.978917,0.740017,0.734474,1.095356
8,DiasABP,5.970082,6.212094,5.733467,4.111301,6.076442,5.709868,5.993303,5.033905,6.660659,6.135140,6.467463,6.936795,5.752702,5.436082,5.417622,6.252846,5.482837
9,FiO2,0.124974,0.120675,0.133841,0.000000,0.125045,0.135671,0.126177,0.124315,0.137327,0.119476,0.119655,0.109538,0.128448,0.117873,0.128707,0.134045,0.124575


<h5>Minimum MAE value in each subgroup</h5>

In [137]:
toolkits.min_value_in_subgroup(df_gpvae_mae_standard_ori, subgroups, variables)

General
MechVent
0.004128255824679912
--------------------
Female
MechVent
0.00408188289412091
--------------------
Male
MechVent
0.003964356557214148
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.0043638204937582565
--------------------
-65
MechVent
0.0036715278702397457
--------------------
ICUType 1
MechVent
0.004535944770565961
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.0038367593567506858
--------------------
ICUType 4
MechVent
0.0037402524419117354
--------------------
Undefined classification
MechVent
0.003936313192582317
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.004750886760606341
--------------------
Overweight
MechVent
0.0043148260716109115
--------------------
Obesity 1
MechVent
0.003922692649272244
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [138]:
toolkits.max_value_in_subgroup(df_gpvae_mae_standard_ori, subgroups, variables)

General
AST
783.051366422483
--------------------
Female
AST
827.2739857074354
--------------------
Male
AST
539.8115787638509
--------------------
Undefined Gender
Urine
67.18858337395625
--------------------
+65
AST
483.56167007420896
--------------------
-65
AST
703.4141769579417
--------------------
ICUType 1
AST
838.3367910003327
--------------------
ICUType 2
ALT
566.6843992534139
--------------------
ICUType 3
AST
738.6059103326402
--------------------
ICUType 4
AST
792.4728483026533
--------------------
Undefined classification
ALT
481.55502367622637
--------------------
Low Weight
AST
1137.682479858114
--------------------
Normal Weight
AST
534.526316687448
--------------------
Overweight
AST
831.4828607467982
--------------------
Obesity 1
AST
535.1885688781471
--------------------
Obesity 2
ALT
853.4166772023752
--------------------
Obesity 3
ALT
1551.118911742647
--------------------


<h4>MinMax Scaler (C/Normalização)</h4>

In [139]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_minmax, subgroups, variables)

GPVAE - MAE
************
General
-------------
ALP : 0.040219907134693526
ALT : 0.017229766448887466
AST : 0.015468583460460567
Albumin : 0.45373933730150834
BUN : 0.11917518228086253
Bilirubin : 0.04461143951106635
Cholesterol : 0.38712575286624884
Creatinine : 0.06330636563215755
DiasABP : 0.2222883897032941
FiO2 : 0.4212544503171473
GCS : 0.7013794357043793
Glucose : 0.08704999971751116
HCO3 : 0.38295172317766624
HCT : 0.45079366319242514
HR : 0.43347807748668743
K : 0.12318451615964963
Lactate : 0.08926922807197073
MAP : 0.2694956399200512
MechVent : 0.0
Mg : 0.06937552593284982
NIDiasABP : 0.34968573475266584
NIMAP : 0.34092245932783305
NISysABP : 0.45429145035202567
Na : 0.4986517947522833
PaCO2 : 0.4095278180110242
PaO2 : 0.29658351907452635
Platelets : 0.08119226622028226
RespRate : 0.19828358249685923
SaO2 : 0.9685085946654024
SysABP : 0.42148474257749563
Temp : 0.9335184893617962
TroponinI : 0.16517311730421255
TroponinT : 0.02879763797807984
Urine : 0.010554451556121645
WBC 

In [140]:
df_gpvae_mae_minmax = toolkits.create_table(testing_mae_gpvae_variables_minmax, subgroups, variables)

In [141]:
df_gpvae_mae_minmax

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.040220,0.041253,0.042669,0.000000,0.045370,0.058368,0.029260,0.018007,0.063298,0.040524,0.036990,0.128495,0.039369,0.032560,0.033538,0.019912,0.031346
1,ALT,0.017230,0.006941,0.016567,0.000000,0.012877,0.023744,0.018795,0.029335,0.023352,0.016529,0.017182,0.012010,0.009226,0.037875,0.014711,0.045587,0.099730
2,AST,0.015469,0.017081,0.006773,0.000000,0.006469,0.011712,0.018681,0.008078,0.013382,0.017210,0.007568,0.024316,0.004764,0.018236,0.010571,0.002253,0.001133
3,Albumin,0.453739,0.423997,0.450085,0.000000,0.428799,0.450342,0.544186,0.392027,0.419730,0.411677,0.404592,0.488372,0.380661,0.470012,0.461240,0.406977,0.390698
4,BUN,0.119175,0.115023,0.134813,0.000000,0.147150,0.102920,0.136926,0.092971,0.158645,0.091490,0.125755,0.143510,0.128078,0.122560,0.140897,0.149504,0.141484
5,Bilirubin,0.044611,0.034648,0.036118,0.000000,0.026320,0.053072,0.018808,0.037440,0.039132,0.045804,0.044419,0.133333,0.055639,0.025541,0.041761,0.026481,0.035695
6,Cholesterol,0.387126,0.473054,0.389912,0.000000,0.363024,0.238024,0.430447,0.000000,0.300150,0.356287,0.427146,0.377246,0.258982,0.533790,0.311377,0.407186,0.000000
7,Creatinine,0.063306,0.048783,0.072079,0.000000,0.057820,0.060337,0.079545,0.051091,0.086067,0.046976,0.065030,0.041558,0.068246,0.056569,0.057444,0.059394,0.076540
8,DiasABP,0.222288,0.221258,0.223552,0.229701,0.212173,0.235335,0.217357,0.208378,0.226277,0.231468,0.230916,0.218497,0.215240,0.216017,0.220501,0.222197,0.221234
9,FiO2,0.421254,0.407955,0.441474,0.000000,0.431625,0.422815,0.415464,0.454358,0.438086,0.399443,0.399171,0.374807,0.413956,0.412962,0.471615,0.444020,0.479529


<h5>Minimum MAE value in each subgroup</h5>

In [142]:
toolkits.min_value_in_subgroup(df_gpvae_mae_minmax, subgroups, variables)

General
MechVent
0.0
--------------------
Female
MechVent
0.0
--------------------
Male
MechVent
0.0
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.0
--------------------
-65
MechVent
0.0
--------------------
ICUType 1
MechVent
0.0
--------------------
ICUType 2
MechVent
0.0
--------------------
ICUType 3
MechVent
0.0
--------------------
ICUType 4
MechVent
0.0
--------------------
Undefined classification
MechVent
0.0
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.0
--------------------
Overweight
MechVent
0.0
--------------------
Obesity 1
MechVent
0.0
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
MechVent
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [143]:
toolkits.max_value_in_subgroup(df_gpvae_mae_minmax, subgroups, variables)

General
SaO2
0.9685085946654024
--------------------
Female
SaO2
0.9617415842380362
--------------------
Male
SaO2
0.9675699026353866
--------------------
Undefined Gender
GCS
0.9375000149009268
--------------------
+65
SaO2
0.9675195857723378
--------------------
-65
SaO2
0.9603608350471073
--------------------
ICUType 1
SaO2
0.964683557990219
--------------------
ICUType 2
SaO2
0.9694730503594138
--------------------
ICUType 3
Temp
0.9274873339532732
--------------------
ICUType 4
SaO2
0.9657142966512496
--------------------
Undefined classification
SaO2
0.9598484974015699
--------------------
Low Weight
SaO2
0.966071435383319
--------------------
Normal Weight
SaO2
0.9706250113936478
--------------------
Overweight
SaO2
0.9674803281393501
--------------------
Obesity 1
SaO2
0.9677868954470741
--------------------
Obesity 2
SaO2
0.962777776850541
--------------------
Obesity 3
SaO2
0.9603703794655621
--------------------


<h4>MinMax Scaler (S/Normalização)</h4>

In [144]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_minmax_ori, subgroups, variables)

GPVAE - MAE
************
General
-------------
ALP : 120.21354016661581
ALT : 292.51042278607537
AST : 566.9945693689813
Albumin : 2.9510791413217787
BUN : 25.788438117812007
Bilirubin : 2.7703703989979305
Cholesterol : 157.29999999999214
Creatinine : 1.4927400290198274
DiasABP : 59.684729736472974
FiO2 : 0.5427910102481897
GCS : 11.416553277655188
Glucose : 139.70664855678496
HCO3 : 22.998730468265872
HCT : 30.609207015520653
HR : 86.69561557043218
K : 4.13614858243916
Lactate : 2.767346140115878
MAP : 80.30970073528962
MechVent : 0.9999999999999996
Mg : 2.011890264982128
NIDiasABP : 58.44698860910438
NIMAP : 77.73032076927599
NISysABP : 119.4786514127118
Na : 138.88944723618073
PaCO2 : 40.95278192858044
PaO2 : 148.29175956176658
Platelets : 190.6867094356714
RespRate : 19.431791012219442
SaO2 : 96.8508583690985
SysABP : 120.12315150566705
Temp : 37.09088744652194
TroponinI : 8.210000050067492
TroponinT : 0.8710493570492538
Urine : 116.09896736077816
WBC : 12.26488773373189
Weight : 8

In [145]:
df_gpvae_mae_minmax_ori = toolkits.create_table(testing_mae_gpvae_variables_minmax_ori, subgroups, variables)

In [146]:
df_gpvae_mae_minmax_ori

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,120.213540,123.095891,127.046296,0.000000,134.583333,170.847619,89.636364,58.240000,184.601941,121.060607,111.202248,366.500000,117.838710,98.843750,101.571429,63.555556,95.454543
1,ALT,292.510423,118.436620,281.300001,0.000000,218.868130,402.716417,319.000002,497.315789,396.091955,280.655731,291.708863,204.200000,157.095240,641.805538,249.900001,772.285714,1688.333333
2,AST,566.994569,625.684209,250.518520,0.000000,239.448055,430.263396,683.920050,298.000001,491.054948,630.363636,279.442109,889.000068,177.375000,667.714272,388.750004,86.000001,45.250000
3,Albumin,2.951079,2.823188,2.935366,0.000000,2.843836,2.936471,3.340000,2.685714,2.804839,2.770213,2.739744,3.100000,2.636842,3.021053,2.983333,2.750000,2.680000
4,BUN,25.788438,24.924859,29.041096,0.000000,31.607143,22.407408,29.480620,20.337931,33.998246,20.029915,27.157143,30.850000,27.640288,26.492424,30.306667,32.096774,30.428572
5,Bilirubin,2.770370,2.151613,2.242935,0.000000,1.634444,3.295798,1.168000,2.325000,2.430097,2.844444,2.758427,8.280000,3.455172,1.586111,2.593333,1.644445,2.216667
6,Cholesterol,157.300000,185.999994,158.230769,0.000000,149.250000,107.500000,171.769229,0.000000,128.250000,147.000000,170.666665,154.000000,114.499992,206.285712,132.000000,163.999985,0.000000
7,Creatinine,1.492740,1.173219,1.685744,0.000000,1.372038,1.427404,1.850000,1.224000,1.993464,1.133466,1.530662,1.014286,1.601408,1.344526,1.363768,1.406667,1.783871
8,DiasABP,59.684730,59.403303,60.029592,61.708333,56.923331,63.246397,58.338428,55.887238,60.773501,62.190792,62.040018,58.649750,57.760640,57.972569,59.196723,59.659678,59.396970
9,FiO2,0.542791,0.532284,0.558764,0.000000,0.550984,0.544024,0.538217,0.568943,0.556088,0.525560,0.525345,0.506098,0.537025,0.536240,0.582576,0.560776,0.588828


<h5>Minimum MAE value in each subgroup</h5>

In [147]:
toolkits.min_value_in_subgroup(df_gpvae_mae_minmax_ori, subgroups, variables)

General
FiO2
0.5427910102481897
--------------------
Female
FiO2
0.5322841955806464
--------------------
Male
FiO2
0.558764148292676
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
FiO2
0.5509840135837525
--------------------
-65
FiO2
0.5440240574521423
--------------------
ICUType 1
FiO2
0.5382165582316663
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
TroponinT
0.4359321938985411
--------------------
ICUType 4
TroponinT
0.4575000104183762
--------------------
Undefined classification
FiO2
0.5253448272903851
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
FiO2
0.5370249722019451
--------------------
Overweight
FiO2
0.5362397804247246
--------------------
Obesity 1
FiO2
0.5825757534816982
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [148]:
toolkits.max_value_in_subgroup(df_gpvae_mae_minmax_ori, subgroups, variables)

General
AST
566.9945693689813
--------------------
Female
AST
625.6842090706995
--------------------
Male
ALT
281.3000011026836
--------------------
Undefined Gender
Urine
139.99999999986
--------------------
+65
AST
239.44805549026773
--------------------
-65
AST
430.2633963482682
--------------------
ICUType 1
AST
683.9200496291841
--------------------
ICUType 2
ALT
497.31578862037054
--------------------
ICUType 3
AST
491.05494833516013
--------------------
ICUType 4
AST
630.3636355139878
--------------------
Undefined classification
ALT
291.7088629807061
--------------------
Low Weight
AST
889.0000677106542
--------------------
Normal Weight
AST
177.37499982118052
--------------------
Overweight
AST
667.7142724082266
--------------------
Obesity 1
AST
388.75000405309646
--------------------
Obesity 2
ALT
772.2857140131256
--------------------
Obesity 3
ALT
1688.3333333327705
--------------------


#### MRNN

<h4> Standard Scaler (C/Normalização)</h4>

In [149]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_standard, subgroups, variables)

MRNN - MAE
************
General
-------------
ALP : 0.5951339004649241
ALT : 0.4759319362481534
AST : 0.5734588221475978
Albumin : 0.728766160074847
BUN : 0.6580606157833552
Bilirubin : 0.5834178022338993
Cholesterol : 0.9309220008663778
Creatinine : 0.6060394273372346
DiasABP : 0.6758033636864339
FiO2 : 0.8088568812039392
GCS : 0.8446816073823585
Glucose : 0.63943468444053
HCO3 : 0.7482965217486706
HCT : 0.7764649646147754
HR : 0.7954725905180468
K : 0.7331447651108022
Lactate : 0.69653545023456
MAP : 0.5973414925028566
MechVent : 0.0942279587803489
Mg : 0.6416707078280048
NIDiasABP : 0.6867621247100877
NIMAP : 0.6277191925813375
NISysABP : 0.7261653549234981
Na : 0.7074724269467934
PaCO2 : 0.7228448454581877
PaO2 : 0.767084791143468
Platelets : 0.7586577660307926
RespRate : 0.7992458489912618
SaO2 : 0.4182720393568836
SysABP : 0.7232698293453811
Temp : 0.5212927791798885
TroponinI : 0.6894572113438211
TroponinT : 0.5133017981077626
Urine : 0.6161699238910668
WBC : 0.17918280815168783

In [150]:
df_mrnn_mae_standard = toolkits.create_table(testing_mae_mrnn_variables_standard, subgroups, variables)

In [151]:
df_mrnn_mae_standard 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.595134,0.569928,0.686871,0.000000,0.769126,1.187601,0.374122,0.544289,1.209770,0.587982,0.530216,2.285989,0.630113,0.441939,0.417916,0.538576,0.311782
1,ALT,0.475932,0.376655,0.403377,0.000000,0.445988,0.541843,0.555393,0.600981,0.603851,0.439216,0.487639,0.432154,0.383987,0.625581,0.398186,0.675480,1.514844
2,AST,0.573459,0.569493,0.389375,0.000000,0.427513,0.425954,0.657688,0.404096,0.547831,0.488531,0.402288,0.599853,0.395957,0.526637,0.416726,0.385794,0.433166
3,Albumin,0.728766,0.747290,0.767402,0.000000,0.676372,0.775963,0.731831,0.938176,0.879837,0.933758,0.938253,0.740743,0.884153,0.787710,0.692587,0.762790,0.845706
4,BUN,0.658061,0.667848,0.678114,0.000000,0.679835,0.718126,0.599659,0.601831,0.734682,0.666382,0.676284,0.566209,0.690837,0.701116,0.670087,0.839600,0.756501
5,Bilirubin,0.583418,0.569861,0.491130,0.000000,0.463989,0.615856,0.464067,0.465251,0.547674,0.569079,0.620235,1.618391,0.687578,0.493451,0.497076,0.562267,0.436497
6,Cholesterol,0.930922,1.317127,0.840432,0.000000,0.673421,1.184312,0.908208,0.000000,0.703393,0.694296,0.579855,0.098861,0.941972,1.319557,0.593558,0.109325,0.000000
7,Creatinine,0.606039,0.544616,0.624639,0.000000,0.498508,0.619633,0.573539,0.500371,0.749812,0.532216,0.675962,0.576992,0.675649,0.541852,0.490919,0.484844,0.593493
8,DiasABP,0.675803,0.700213,0.634305,0.269653,0.696885,0.615245,0.723857,0.603417,0.713862,0.645980,0.684188,0.654811,0.671089,0.647798,0.612158,0.673746,0.632342
9,FiO2,0.808857,0.826696,0.809285,0.000000,0.807161,0.845317,0.819277,0.758587,0.872102,0.797893,0.832479,0.876523,0.798787,0.745886,0.763620,0.894003,0.753385


<h5>Minimum MAE value in each subgroup</h5>

In [152]:
toolkits.min_value_in_subgroup(df_mrnn_mae_standard, subgroups, variables)

General
MechVent
0.0942279587803489
--------------------
Female
MechVent
0.09341611247126444
--------------------
Male
MechVent
0.09514540282531944
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.09347160937063184
--------------------
-65
MechVent
0.09416960407486218
--------------------
ICUType 1
MechVent
0.09836334022257755
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.09485922366953797
--------------------
ICUType 4
MechVent
0.09281785016985615
--------------------
Undefined classification
MechVent
0.09269475338146597
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.09370095254224839
--------------------
Overweight
MechVent
0.09742545178541286
--------------------
Obesity 1
MechVent
0.09285577279592695
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [153]:
toolkits.max_value_in_subgroup(df_mrnn_mae_standard, subgroups, variables)

General
Cholesterol
0.9309220008663778
--------------------
Female
Cholesterol
1.317127244116942
--------------------
Male
GCS
0.8667862204143494
--------------------
Undefined Gender
NISysABP
1.0938912821593096
--------------------
+65
TroponinI
0.9342441204653614
--------------------
-65
ALP
1.1876005572858588
--------------------
ICUType 1
TroponinI
1.3081553001030344
--------------------
ICUType 2
Albumin
0.9381759076218124
--------------------
ICUType 3
SaO2
1.7068288892332344
--------------------
ICUType 4
Albumin
0.9337579107938018
--------------------
Undefined classification
Albumin
0.9382526424008452
--------------------
Low Weight
ALP
2.285989103089192
--------------------
Normal Weight
Cholesterol
0.9419718811850725
--------------------
Overweight
Cholesterol
1.3195565058919039
--------------------
Obesity 1
Ph
1.0026911371029357
--------------------
Obesity 2
TroponinT
1.3698709231174193
--------------------
Obesity 3
Weight
1.697952667789286
--------------------


<h4> Standard Scaler (S/Normalização)</h4>

In [154]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_standard_ori, subgroups, variables)

MRNN - MAE
************
General
-------------
ALP : 77.09593296050986
ALT : 558.4894344011914
AST : 1006.3577661928869
Albumin : 0.47858002957679896
BUN : 15.03467769552419
Bilirubin : 3.3424833413784922
Cholesterol : 41.47522125243933
Creatinine : 0.9345179706044152
DiasABP : 8.642677431777495
FiO2 : 0.15205061671407266
GCS : 3.3795056063960747
Glucose : 40.92191259988434
HCO3 : 3.538717524049241
HCT : 3.8964098408568772
HR : 14.144704845683677
K : 0.49734669193491154
Lactate : 1.8522080101936444
MAP : 9.472260047770028
MechVent : 0.09422795753116188
Mg : 0.31147617677362915
NIDiasABP : 10.299804339405927
NIMAP : 9.350919729784227
NISysABP : 16.104638767484147
Na : 3.711160554358703
PaCO2 : 6.590632414922529
PaO2 : 65.46823137109153
Platelets : 81.63416715886302
RespRate : 4.290442534104192
SaO2 : 1.4387517609821336
SysABP : 16.9487512128442
Temp : 0.6652028258526668
TroponinI : 7.637945709228134
TroponinT : 1.5456071574305215
Urine : 96.38054251687838
WBC : 7.401536362671937
Weight :

In [155]:
df_mrnn_mae_standard_ori = toolkits.create_table(testing_mae_mrnn_variables_standard_ori, subgroups, variables)

In [156]:
df_mrnn_mae_standard_ori

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,77.095933,73.830672,88.979881,0.000000,99.635556,153.846341,48.465203,70.509316,156.718262,76.169397,68.686156,296.135818,81.627247,57.250508,54.138482,69.769257,40.389494
1,ALT,558.489434,441.990923,473.348472,0.000000,523.350797,635.833363,651.734232,705.230149,708.597617,515.404113,572.227610,507.117865,450.595388,734.097596,467.257407,792.651703,1777.615885
2,AST,1006.357766,999.398716,683.311014,0.000000,750.238200,747.503545,1154.170000,709.144815,961.383529,857.319265,705.972131,1052.677109,694.860899,924.190863,731.308066,677.027018,760.158997
3,Albumin,0.478580,0.490744,0.503952,0.000000,0.444173,0.509574,0.480593,0.616099,0.577788,0.613198,0.616150,0.486445,0.580622,0.517288,0.454821,0.500923,0.555375
4,BUN,15.034678,15.258285,15.492828,0.000000,15.532151,16.406986,13.700370,13.750000,16.785238,15.224796,15.451035,12.936158,15.783512,16.018371,15.309451,19.182298,17.283729
5,Bilirubin,3.342483,3.264814,2.813752,0.000000,2.658261,3.528326,2.658704,2.665491,3.137702,3.260333,3.553417,9.271992,3.939235,2.827051,2.847820,3.221305,2.500753
6,Cholesterol,41.475221,58.681763,37.443616,0.000000,30.002832,52.764481,40.463253,0.000000,31.338181,30.932866,25.834173,4.404556,41.967522,58.789989,26.444682,4.870728,0.000000
7,Creatinine,0.934518,0.839803,0.963199,0.000000,0.768703,0.955479,0.884403,0.771577,1.156216,0.820681,1.042340,0.889727,1.041857,0.835540,0.757002,0.747634,0.915171
8,DiasABP,8.642677,8.954849,8.111965,3.448525,8.912290,7.868218,9.257222,7.716941,9.129405,8.261272,8.749901,8.374218,8.582388,8.284525,7.828733,8.616370,8.086860
9,FiO2,0.152051,0.155404,0.152131,0.000000,0.151732,0.158904,0.154009,0.142601,0.163940,0.149990,0.156491,0.164771,0.150158,0.140213,0.143547,0.168057,0.141623


<h5>Minimum MAE value in each subgroup</h5>

In [157]:
toolkits.min_value_in_subgroup(df_mrnn_mae_standard_ori, subgroups, variables)

General
MechVent
0.09422795753116188
--------------------
Female
MechVent
0.09341611636518929
--------------------
Male
MechVent
0.09514540077584362
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.09347161274772496
--------------------
-65
MechVent
0.09416960241607795
--------------------
ICUType 1
MechVent
0.09836334118740094
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.09485922489676571
--------------------
ICUType 4
MechVent
0.092817851134184
--------------------
Undefined classification
MechVent
0.09269475217798746
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.09370095305647558
--------------------
Overweight
MechVent
0.09742545314699556
--------------------
Obesity 1
MechVent
0.09285577818833611
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [158]:
toolkits.max_value_in_subgroup(df_mrnn_mae_standard_ori, subgroups, variables)

General
AST
1006.3577661928869
--------------------
Female
AST
999.3987161736609
--------------------
Male
AST
683.3110142460513
--------------------
Undefined Gender
NISysABP
24.259933471667555
--------------------
+65
AST
750.2381996055606
--------------------
-65
AST
747.5035454886233
--------------------
ICUType 1
AST
1154.169999999954
--------------------
ICUType 2
AST
709.1448151961309
--------------------
ICUType 3
AST
961.3835288246804
--------------------
ICUType 4
AST
857.3192649147571
--------------------
Undefined classification
AST
705.9721313476489
--------------------
Low Weight
AST
1052.6771087643851
--------------------
Normal Weight
AST
694.8608989715359
--------------------
Overweight
AST
924.1908627464219
--------------------
Obesity 1
AST
731.3080657958619
--------------------
Obesity 2
ALT
792.6517028807461
--------------------
Obesity 3
ALT
1777.615885416074
--------------------


<h4>MinMax Scaler (C/Normalização)</h4>

In [159]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_minmax, subgroups, variables)

MRNN - MAE
************
General
-------------
ALP : 0.040219907134693526
ALT : 0.017229766448887466
AST : 0.015468583460460567
Albumin : 0.45373933730150834
BUN : 0.11917518228086253
Bilirubin : 0.04461143951106635
Cholesterol : 0.38712575286624884
Creatinine : 0.06330636563215755
DiasABP : 0.2222883897032941
FiO2 : 0.4212544503171473
GCS : 0.7013794357043793
Glucose : 0.08704999971751116
HCO3 : 0.38295172317766624
HCT : 0.45079366319242514
HR : 0.43347807748668743
K : 0.12318451615964963
Lactate : 0.08926922807197073
MAP : 0.2694956399200512
MechVent : 0.0
Mg : 0.06937552593284982
NIDiasABP : 0.34968573475266584
NIMAP : 0.34092245932783305
NISysABP : 0.45429145035202567
Na : 0.4986517947522833
PaCO2 : 0.4095278180110242
PaO2 : 0.29658351907452635
Platelets : 0.08119226622028226
RespRate : 0.19828358249685923
SaO2 : 0.9685085946654024
SysABP : 0.42148474257749563
Temp : 0.9335184893617962
TroponinI : 0.16517311730421255
TroponinT : 0.02879763797807984
Urine : 0.010554451556121645
WBC :

In [160]:
df_mrnn_mae_minmax = toolkits.create_table(testing_mae_mrnn_variables_minmax, subgroups, variables)

In [161]:
df_mrnn_mae_minmax

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.040220,0.041253,0.042669,0.000000,0.045370,0.058368,0.029260,0.018007,0.063298,0.040524,0.036990,0.128495,0.039369,0.032560,0.033538,0.019912,0.031346
1,ALT,0.017230,0.006941,0.016567,0.000000,0.012877,0.023744,0.018795,0.029335,0.023352,0.016529,0.017182,0.012010,0.009226,0.037875,0.014711,0.045587,0.099730
2,AST,0.015469,0.017081,0.006773,0.000000,0.006469,0.011712,0.018681,0.008078,0.013382,0.017210,0.007568,0.024316,0.004764,0.018236,0.010571,0.002253,0.001133
3,Albumin,0.453739,0.423997,0.450085,0.000000,0.428799,0.450342,0.544186,0.392027,0.419730,0.411677,0.404592,0.488372,0.380661,0.470012,0.461240,0.406977,0.390698
4,BUN,0.119175,0.115023,0.134813,0.000000,0.147150,0.102920,0.136926,0.092971,0.158645,0.091490,0.125755,0.143510,0.128078,0.122560,0.140897,0.149504,0.141484
5,Bilirubin,0.044611,0.034648,0.036118,0.000000,0.026320,0.053072,0.018808,0.037440,0.039132,0.045804,0.044419,0.133333,0.055639,0.025541,0.041761,0.026481,0.035695
6,Cholesterol,0.387126,0.473054,0.389912,0.000000,0.363024,0.238024,0.430447,0.000000,0.300150,0.356287,0.427146,0.377246,0.258982,0.533790,0.311377,0.407186,0.000000
7,Creatinine,0.063306,0.048783,0.072079,0.000000,0.057820,0.060337,0.079545,0.051091,0.086067,0.046976,0.065030,0.041558,0.068246,0.056569,0.057444,0.059394,0.076540
8,DiasABP,0.222288,0.221258,0.223552,0.229701,0.212173,0.235335,0.217357,0.208378,0.226277,0.231468,0.230916,0.218497,0.215240,0.216017,0.220501,0.222197,0.221234
9,FiO2,0.421254,0.407955,0.441474,0.000000,0.431625,0.422815,0.415464,0.454358,0.438086,0.399443,0.399171,0.374807,0.413956,0.412962,0.471615,0.444020,0.479529


<h5>Minimum MAE value in each subgroup</h5>

In [162]:
toolkits.min_value_in_subgroup(df_mrnn_mae_minmax, subgroups, variables)

General
MechVent
0.0
--------------------
Female
MechVent
0.0
--------------------
Male
MechVent
0.0
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.0
--------------------
-65
MechVent
0.0
--------------------
ICUType 1
MechVent
0.0
--------------------
ICUType 2
MechVent
0.0
--------------------
ICUType 3
MechVent
0.0
--------------------
ICUType 4
MechVent
0.0
--------------------
Undefined classification
MechVent
0.0
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.0
--------------------
Overweight
MechVent
0.0
--------------------
Obesity 1
MechVent
0.0
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
MechVent
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [163]:
toolkits.max_value_in_subgroup(df_mrnn_mae_minmax, subgroups, variables)

General
SaO2
0.9685085946654024
--------------------
Female
SaO2
0.9617415842380362
--------------------
Male
SaO2
0.9675699026353866
--------------------
Undefined Gender
GCS
0.9375000149009268
--------------------
+65
SaO2
0.9675195857723378
--------------------
-65
SaO2
0.9603608350471073
--------------------
ICUType 1
SaO2
0.964683557990219
--------------------
ICUType 2
SaO2
0.9694730503594138
--------------------
ICUType 3
Temp
0.9274873339532732
--------------------
ICUType 4
SaO2
0.9657142966512496
--------------------
Undefined classification
SaO2
0.9598484974015699
--------------------
Low Weight
SaO2
0.966071435383319
--------------------
Normal Weight
SaO2
0.9706250113936478
--------------------
Overweight
SaO2
0.9674803281393501
--------------------
Obesity 1
SaO2
0.9677868954470741
--------------------
Obesity 2
SaO2
0.962777776850541
--------------------
Obesity 3
SaO2
0.9603703794655621
--------------------


<h4>MinMax Scaler (S/Normalização)</h4>

In [164]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_minmax_ori, subgroups, variables)

MRNN - MAE
************
General
-------------
ALP : 120.21354016661581
ALT : 292.51042278607537
AST : 566.9945693689813
Albumin : 2.9510791413217787
BUN : 25.788438117812007
Bilirubin : 2.7703703989979305
Cholesterol : 157.29999999999214
Creatinine : 1.4927400290198274
DiasABP : 59.684729736472974
FiO2 : 0.5427910102481897
GCS : 11.416553277655188
Glucose : 139.70664855678496
HCO3 : 22.998730468265872
HCT : 30.609207015520653
HR : 86.69561557043218
K : 4.13614858243916
Lactate : 2.767346140115878
MAP : 80.30970073528962
MechVent : 0.9999999999999996
Mg : 2.011890264982128
NIDiasABP : 58.44698860910438
NIMAP : 77.73032076927599
NISysABP : 119.4786514127118
Na : 138.88944723618073
PaCO2 : 40.95278192858044
PaO2 : 148.29175956176658
Platelets : 190.6867094356714
RespRate : 19.431791012219442
SaO2 : 96.8508583690985
SysABP : 120.12315150566705
Temp : 37.09088744652194
TroponinI : 8.210000050067492
TroponinT : 0.8710493570492538
Urine : 116.09896736077816
WBC : 12.26488773373189
Weight : 82

In [165]:
df_mrnn_mae_minmax_ori = toolkits.create_table(testing_mae_mrnn_variables_minmax_ori, subgroups, variables)

In [166]:
df_mrnn_mae_minmax_ori 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,120.213540,123.095891,127.046296,0.000000,134.583333,170.847619,89.636364,58.240000,184.601941,121.060607,111.202248,366.500000,117.838710,98.843750,101.571429,63.555556,95.454543
1,ALT,292.510423,118.436620,281.300001,0.000000,218.868130,402.716417,319.000002,497.315789,396.091955,280.655731,291.708863,204.200000,157.095240,641.805538,249.900001,772.285714,1688.333333
2,AST,566.994569,625.684209,250.518520,0.000000,239.448055,430.263396,683.920050,298.000001,491.054948,630.363636,279.442109,889.000068,177.375000,667.714272,388.750004,86.000001,45.250000
3,Albumin,2.951079,2.823188,2.935366,0.000000,2.843836,2.936471,3.340000,2.685714,2.804839,2.770213,2.739744,3.100000,2.636842,3.021053,2.983333,2.750000,2.680000
4,BUN,25.788438,24.924859,29.041096,0.000000,31.607143,22.407408,29.480620,20.337931,33.998246,20.029915,27.157143,30.850000,27.640288,26.492424,30.306667,32.096774,30.428572
5,Bilirubin,2.770370,2.151613,2.242935,0.000000,1.634444,3.295798,1.168000,2.325000,2.430097,2.844444,2.758427,8.280000,3.455172,1.586111,2.593333,1.644445,2.216667
6,Cholesterol,157.300000,185.999994,158.230769,0.000000,149.250000,107.500000,171.769229,0.000000,128.250000,147.000000,170.666665,154.000000,114.499992,206.285712,132.000000,163.999985,0.000000
7,Creatinine,1.492740,1.173219,1.685744,0.000000,1.372038,1.427404,1.850000,1.224000,1.993464,1.133466,1.530662,1.014286,1.601408,1.344526,1.363768,1.406667,1.783871
8,DiasABP,59.684730,59.403303,60.029592,61.708333,56.923331,63.246397,58.338428,55.887238,60.773501,62.190792,62.040018,58.649750,57.760640,57.972569,59.196723,59.659678,59.396970
9,FiO2,0.542791,0.532284,0.558764,0.000000,0.550984,0.544024,0.538217,0.568943,0.556088,0.525560,0.525345,0.506098,0.537025,0.536240,0.582576,0.560776,0.588828


<h5>Minimum MAE value in each subgroup</h5>

In [167]:
toolkits.min_value_in_subgroup(df_mrnn_mae_minmax_ori, subgroups, variables)

General
FiO2
0.5427910102481897
--------------------
Female
FiO2
0.5322841955806464
--------------------
Male
FiO2
0.558764148292676
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
FiO2
0.5509840135837525
--------------------
-65
FiO2
0.5440240574521423
--------------------
ICUType 1
FiO2
0.5382165582316663
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
TroponinT
0.4359321938985411
--------------------
ICUType 4
TroponinT
0.4575000104183762
--------------------
Undefined classification
FiO2
0.5253448272903851
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
FiO2
0.5370249722019451
--------------------
Overweight
FiO2
0.5362397804247246
--------------------
Obesity 1
FiO2
0.5825757534816982
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [168]:
toolkits.max_value_in_subgroup(df_mrnn_mae_minmax_ori, subgroups, variables)

General
AST
566.9945693689813
--------------------
Female
AST
625.6842090706995
--------------------
Male
ALT
281.3000011026836
--------------------
Undefined Gender
Urine
139.99999999986
--------------------
+65
AST
239.44805549026773
--------------------
-65
AST
430.2633963482682
--------------------
ICUType 1
AST
683.9200496291841
--------------------
ICUType 2
ALT
497.31578862037054
--------------------
ICUType 3
AST
491.05494833516013
--------------------
ICUType 4
AST
630.3636355139878
--------------------
Undefined classification
ALT
291.7088629807061
--------------------
Low Weight
AST
889.0000677106542
--------------------
Normal Weight
AST
177.37499982118052
--------------------
Overweight
AST
667.7142724082266
--------------------
Obesity 1
AST
388.75000405309646
--------------------
Obesity 2
ALT
772.2857140131256
--------------------
Obesity 3
ALT
1688.3333333327705
--------------------
